# Organização geral do tratamento dos dados da CGU

**Resumo do que está no Google Big Query (GBQ)**
- Militares 
    - ativos (Padrão: ano_mes_militares_ativos_cadastro/remuneracao)
        - Cadastro: No GBQ (Duplicado em ID e NOME + CPF) 
        - Remuneração: No GBQ

    - reforma/reserva: (Padrão: ano_mes_militares_reserva_reforma_cadastro/remuneracao)
        - Cadastro: No GBQ (Duplicado)
        - Remuneração: No GBQ 

    - Pensionistas (Padrão: ano_mes_militares_pensionistas_cadastro/remuneracao)
        - Cadastro: No GBQ (Duplicado)
        - Remuneração: No GBQ (Duplicado)
- SIAPE
    - ativos (Padrão: ano_mes_siape_servidores_cadastro/remuneracao)
        - Cadastro: No GBQ (Duplicado)
        - Remuneração: No GBQ

    - aposentados (Padrão: ano_mes_siape_aposentados_cadastro/remuneracao)
        - Cadastro: No GBQ (Duplicado em ID e NOME + CPF)
        - Remuneração: No GBQ 

    - pensionistas (Padrão: ano_mes_siape_pensionista_cadastro/remuneracao)
        - Cadastro: No GBQ (Duplicado em ID e NOME + CPF)
        - Remuneração: No GBQ (Duplicado em ID e NOME + CPF)
- BACEN
    - ativos (Padrão: ano_mes_bacen_servidores_cadastro/remuneracao)
        - Cadastro: No GBQ (Duplicado em ID e NOME + CPF)
        - Remuneração: No GBQ

    - aposentados (Padrão: ano_mes_bacen_aposentados_cadastro/remuneracao)
        - Cadastro: No GBQ
        - Remuneração: No GBQ
         
    - pensionistas (Padrão: ano_mes_bacen_pensionistas_cadastro/remuneracao)
        - Cadastro: No GBQ (Duplicado em ID e NOME + CPF)
        - Remuneração: No GBQ (Duplicado em ID e NOME + CPF)

Importar pacotes e baixar bibliotecas necessárias para análise

In [1]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import zipfile
import pandas_gbq
from google.cloud import bigquery
import re
import calendar ##importando essa biblioteca pra ter acesso a mes/dias/anos
import locale ##pra configurar o idioma do calendario
import glob
import openpyxl
import shutil
import zipfile

In [63]:
%pip install pyxlsb

  Obtaining dependency information for pyxlsb from https://files.pythonhosted.org/packages/7e/92/345823838ae367c59b63e03aef9c331f485370f9df6d049256a61a28f06d/pyxlsb-1.0.10-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [66]:
%pip install xlrd

  Obtaining dependency information for xlrd from https://files.pythonhosted.org/packages/a6/0c/c2a72d51fe56e08a08acc85d13013558a2d793028ae7385448a6ccdfae64/xlrd-2.0.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
   ---------------------------------------- 96.5/96.5 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [71]:
%pip install --upgrade xlrd


Note: you may need to restart the kernel to use updated packages.


In [72]:
%pip install openpyxl


Note: you may need to restart the kernel to use updated packages.


In [2]:

client = bigquery.Client()


# Loops com biblioteca OS

Unzippin os dados em loop 


In [2]:
#diretório onde os arquivos zip estão localizados
diretorio = 'C:\\Users\\ana.sales_republica\\Downloads\\siape.zips\\ativos'

In [6]:
os.listdir(diretorio)

['201301_Servidores_SIAPE.zip',
 '201302_Servidores_SIAPE.zip',
 '201303_Servidores_SIAPE.zip',
 '201304_Servidores_SIAPE.zip',
 '201305_Servidores_SIAPE.zip',
 '201306_Servidores_SIAPE.zip',
 '201307_Servidores_SIAPE.zip',
 '201308_Servidores_SIAPE.zip',
 '201309_Servidores_SIAPE.zip',
 '201310_Servidores_SIAPE.zip',
 '201311_Servidores_SIAPE.zip',
 '201312_Servidores_SIAPE.zip',
 '201401_Servidores_SIAPE.zip',
 '201402_Servidores_SIAPE.zip',
 '201403_Servidores_SIAPE.zip',
 '201404_Servidores_SIAPE.zip',
 '201405_Servidores_SIAPE.zip',
 '201406_Servidores_SIAPE.zip',
 '201407_Servidores_SIAPE.zip',
 '201408_Servidores_SIAPE.zip',
 '201409_Servidores_SIAPE.zip',
 '201410_Servidores_SIAPE.zip',
 '201411_Servidores_SIAPE.zip',
 '201412_Servidores_SIAPE.zip',
 '201501_Servidores_SIAPE.zip',
 '201502_Servidores_SIAPE.zip',
 '201503_Servidores_SIAPE.zip',
 '201504_Servidores_SIAPE.zip',
 '201505_Servidores_SIAPE.zip',
 '201506_Servidores_SIAPE.zip',
 '201507_Servidores_SIAPE.zip',
 '201508

In [80]:
for item in os.listdir(diretorio): #importante listar aqui e *não* atribuir a lista a uma variável, pois não funcionará em os.path.join
    # Verificando se o arquivo é um arquivo zip
    if item.endswith('.zip'):
        # Construindo o caminho completo do arquivo zip
        caminho_completo = os.path.join(diretorio, item)
        
        # Extrai o conteúdo do arquivo zip
        with zipfile.ZipFile(caminho_completo, 'r') as sem_zip:
            sem_zip.extractall(diretorio)

In [12]:
from pathlib import Path

In [13]:
# Criando uma pasta para Cadastro

Path('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Dados novos\\\\CGU\\\\MILITARES\\\\Militares_reserva\\Cadastro').mkdir(exist_ok=True)


In [14]:
# Criando uma pasta para Remuneração
Path('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Dados novos\\\\CGU\\\\MILITARES\\\\Militares_reserva\\Remuneração').mkdir(exist_ok=True)


Para remover arquivos


In [31]:
os.listdir(diretorio)

['202401_Cadastro.csv',
 '202401_Observacoes.csv',
 '202401_Pensionistas_BACEN.zip',
 '202401_Remuneracao.csv',
 '202402_Cadastro.csv',
 '202402_Observacoes.csv',
 '202402_Pensionistas_BACEN.zip',
 '202402_Remuneracao.csv']

In [6]:
for i in os.listdir(diretorio): #importante listar aqui e *não* atribuir a lista a uma variável, pois não funcionará em os.path.join
         if i.endswith('.zip'):
            os.remove(os.path.join(diretorio, i))

Mudando diretório em loop

In [21]:
os.chdir(diretorio)

In [29]:
diretorio

'C:\\Users\\ana.sales_republica\\Downloads\\bacen zips'

In [35]:
for arquivo in os.listdir(diretorio):
    #Se atender a um padrao (aqui seria '_cadastro' no nome), mudar o diretório.
    if '_Observacoes' in arquivo:
        os.rename(f"C:\\Users\\ana.sales_republica\\Downloads\\bacen zips\\{arquivo}", f"C:\\Users\\ana.sales_republica\\Downloads\\bacen zips\\observacoes\\{arquivo}")
    

In [19]:
#Renomear um arquivo é um metodo da biblioteca os que permite mudar o arquivo de lugar 
for arquivo in os.listdir(diretorio):
    #Se atender a um padrao (aqui seria '_cadastro' no nome), mudar o diretório.
    if '_Cadastro' in arquivo:
        os.rename(f"G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Dados novos\\\\CGU\\\\MILITARES\\Militares_reserva\\{arquivo}", f"G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\Dados novos\\CGU\\MILITARES\\Militares_reserva\\Cadastro\\{arquivo}")
    if '_Remuneracao' in arquivo: ##aqui seria '_Remuneracao' no nome
        os.rename(f"G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Dados novos\\\\CGU\\\\MILITARES\\Militares_reserva\\{arquivo}", f"G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\Dados novos\\CGU\\MILITARES\\Militares_reserva\\Remuneração\\{arquivo}")        

Renomeando em loop

In [11]:
##importando pacotes
import os
import re
import calendar ##importando essa biblioteca pra ter acesso a mes/dias/anos
import locale ##pra configurar o idioma do calendario
import pandas as pd
import numpy as np


In [4]:
locale.setlocale(locale.LC_TIME, 'PT_BR')

'PT_BR'

In [5]:
#definindo uma função para obter o nome abreviado do mês a partir do número do mês
def abreviacao_mes(numero_mes):
    return calendar.month_abbr[int(numero_mes)]

In [45]:
lista = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados novos\\CGU\\BACEN\\SERVIDORES_ATIVOS_REMUNERACAO'

In [83]:
#expressão regular (re) para encontrar os arquivos no formato que eu quero
padrao = re.compile(r'(\d{4})(\d{2})_Cadastro\.csv') ##adaptar de acordo com o arquivo que quero renomear

In [84]:
# Percorrendo todos os arquivos no diretório
for arquivo in os.listdir(lista):
    if padrao.match(arquivo):  # Verifica se o nome do arquivo corresponde ao padrão
        ano, mes = padrao.match(arquivo).groups()  # Extrai o ano e o mês do nome do arquivo
        abrev_mes = abreviacao_mes(mes)  # Obtém a abreviação do mês
        novo_nome = f'{ano}_{abrev_mes}_siape_servidores_cadastro.csv' # Novo nome que quero mudar
        novo_nome = novo_nome.lower() #deixando o nome dos arquivos em letra minúscula
        caminho_antigo = os.path.join(lista, arquivo) # Caminho completo para o arquivo antigo e novo
        caminho_novo = os.path.join(lista, novo_nome)
        os.rename(caminho_antigo, caminho_novo) # Renomeia o arquivo
        print(f'Arquivo renomeado: {arquivo} -> {novo_nome}') 

Arquivo renomeado: 202401_Cadastro.csv -> 2024_jan_siape_servidores_cadastro.csv
Arquivo renomeado: 202402_Cadastro.csv -> 2024_fev_siape_servidores_cadastro.csv
Arquivo renomeado: 202403_Cadastro.csv -> 2024_mar_siape_servidores_cadastro.csv


# LOOP PARA SUBIR PARA O GBQ OS DADOS DE CADASTROS E REMUNERAÇÃO DE ATIVOS, PENSIONISTAS E APOSENTADOS DO SIAPE, BACEN E MILITARES.


In [7]:
# Adequar meses e anos conforme os dados existentes 
anos =  range(2013, 2025) 
meses = ['jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez']


In [14]:
diretorio_cadastro = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Atualização Dados novos\\CGU\\MILITARES\\RESERVA_REFORMA_CADASTRO'

In [15]:
os.chdir(diretorio_cadastro)


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169797 entries, 0 to 169796
Data columns (total 27 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Id_SERVIDOR_PORTAL                    169797 non-null  int64  
 1   NOME                                  169797 non-null  object 
 2   CPF                                   169797 non-null  object 
 3   MATRICULA                             169797 non-null  object 
 4   COD_TIPO_APOSENTADORIA                169797 non-null  int64  
 5   TIPO_APOSENTADORIA                    169797 non-null  object 
 6   DATA_APOSENTADORIA                    169784 non-null  object 
 7   DESCRICAO_CARGO                       169797 non-null  object 
 8   COD_UORG_LOTACAO                      169797 non-null  int64  
 9   UORG_LOTACAO                          169797 non-null  object 
 10  COD_ORG_LOTACAO                       169797 non-null  int64  
 11  

In [32]:
diretorio_remuneracao = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Atualização Dados novos\\CGU\\MILITARES\\RESERVA_REFORMA_REMUNERACAO'
os.chdir(diretorio_remuneracao)
os.listdir(diretorio_remuneracao)

['2023_mai_militares_ativos_remuneracao.csv',
 '2023_jun_militares_ativos_remuneracao.csv',
 '2023_jul_militares_ativos_remuneracao.csv',
 '2023_ago_militares_ativos_remuneracao.csv',
 '2023_set_militares_ativos_remuneracao.csv',
 '2023_out_militares_ativos_remuneracao.csv',
 '2023_nov_militares_ativos_remuneracao.csv',
 '2023_dez_militares_ativos_remuneracao.csv',
 '2013_jan_militares_ativos_remuneracao.csv',
 '2013_fev_militares_ativos_remuneracao.csv',
 '2013_mar_militares_ativos_remuneracao.csv',
 '2013_abr_militares_ativos_remuneracao.csv',
 '2013_mai_militares_ativos_remuneracao.csv',
 '2013_jun_militares_ativos_remuneracao.csv',
 '2013_jul_militares_ativos_remuneracao.csv',
 '2013_ago_militares_ativos_remuneracao.csv',
 '2013_set_militares_ativos_remuneracao.csv',
 '2013_out_militares_ativos_remuneracao.csv',
 '2013_nov_militares_ativos_remuneracao.csv',
 '2013_dez_militares_ativos_remuneracao.csv',
 '2014_jan_militares_ativos_remuneracao.csv',
 '2014_fev_militares_ativos_remune

### MILITARES

In [31]:
##MILITARES ATIVOS. 
for ano in anos:
    for mes_num, mes in enumerate(meses, start=1):
        nome_arquivo = f"{ano}_{mes}_militares_ativos_cadastro.csv"
        caminho_arquivo = os.path.join(diretorio_cadastro, nome_arquivo)

        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            
            # Criar colunas de ano e mês de acordo com o nome do arquivo
            df['ANO'] = ano
            df['MES'] = mes_num

            # Carregando o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_militares_ativos_cadastro"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema= ([bigquery.SchemaField('ANO', 'INTEGER', description=''),
                                bigquery.SchemaField('MES', 'INTEGER', description=''),
                                bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'INTEGER', description=''),
                                bigquery.SchemaField('NOME', 'STRING', description=''),
                                bigquery.SchemaField('CPF', 'STRING', description=''),
                                bigquery.SchemaField('MATRICULA', 'STRING', description=''),
                                bigquery.SchemaField('DESCRICAO_CARGO', 'STRING', description=''),
                                bigquery.SchemaField('CLASSE_CARGO', 'INTEGER', description=''),
                                bigquery.SchemaField('REFERENCIA_CARGO', 'INTEGER', description=''),
                                bigquery.SchemaField('PADRAO_CARGO', 'INTEGER', description=''),
                                bigquery.SchemaField('NIVEL_CARGO', 'INTEGER', description=''),
                                bigquery.SchemaField('SIGLA_FUNCAO', 'INTEGER', description=''),
                                bigquery.SchemaField('NIVEL_FUNCAO', 'INTEGER', description=''),
                                bigquery.SchemaField('FUNCAO', 'STRING', description=''),
                                bigquery.SchemaField('CODIGO_ATIVIDADE', 'INTEGER', description=''),
                                bigquery.SchemaField('ATIVIDADE', 'STRING', description=''),
                                bigquery.SchemaField('OPCAO_PARCIAL', 'FLOAT', description=''),
                                bigquery.SchemaField('COD_UORG_LOTACAO', 'INTEGER', description=''),
                                bigquery.SchemaField('UORG_LOTACAO', 'STRING', description=''),
                                bigquery.SchemaField('COD_ORG_LOTACAO', 'INTEGER', description=''),
                                bigquery.SchemaField('ORG_LOTACAO', 'STRING', description=''),
                                bigquery.SchemaField('COD_ORGSUP_LOTACAO', 'INTEGER', description=''),
                                bigquery.SchemaField('ORGSUP_LOTACAO', 'STRING', description=''),
                                bigquery.SchemaField('COD_UORG_EXERCICIO', 'INTEGER', description=''),
                                bigquery.SchemaField('UORG_EXERCICIO', 'STRING', description=''),
                                bigquery.SchemaField('COD_ORG_EXERCICIO', 'INTEGER', description=''),
                                bigquery.SchemaField('ORG_EXERCICIO', 'STRING', description=''),
                                bigquery.SchemaField('COD_ORGSUP_EXERCICIO', 'INTEGER', description=''),
                                bigquery.SchemaField('ORGSUP_EXERCICIO', 'STRING', description=''),
                                bigquery.SchemaField('COD_TIPO_VINCULO', 'INTEGER', description=''),
                                bigquery.SchemaField('TIPO_VINCULO', 'STRING', description=''),
                                bigquery.SchemaField('SITUACAO_VINCULO', 'STRING', description=''),
                                bigquery.SchemaField('DATA_INICIO_AFASTAMENTO', 'STRING', description=''),
                                bigquery.SchemaField('DATA_TERMINO_AFASTAMENTO', 'STRING', description=''),
                                bigquery.SchemaField('REGIME_JURIDICO', 'STRING', description=''),
                                bigquery.SchemaField('JORNADA_DE_TRABALHO', 'STRING', description=''),
                                bigquery.SchemaField('DATA_INGRESSO_CARGOFUNCAO', 'FLOAT', description=''),
                                bigquery.SchemaField('DATA_NOMEACAO_CARGOFUNCAO', 'FLOAT', description=''),
                                bigquery.SchemaField('DATA_INGRESSO_ORGAO', 'FLOAT', description=''),
                                bigquery.SchemaField('DOCUMENTO_INGRESSO_SERVICOPUBLICO', 'STRING', description=''),
                                bigquery.SchemaField('DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO', 'FLOAT', description=''),
                                bigquery.SchemaField('DIPLOMA_INGRESSO_CARGOFUNCAO', 'FLOAT', description=''),
                                bigquery.SchemaField('DIPLOMA_INGRESSO_ORGAO', 'STRING', description=''),
                                bigquery.SchemaField('DIPLOMA_INGRESSO_SERVICOPUBLICO', 'STRING', description=''),
                                bigquery.SchemaField('UF_EXERCICIO', 'INTEGER', description='')
                                ])
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_7144\1942090255.py:9: DtypeWarning: Columns (30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_jan_militares_ativos_cadastro.csv carregado para a tabela 2024_jan_militares_ativos_cadastro no BigQuery.


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_7144\1942090255.py:9: DtypeWarning: Columns (30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_fev_militares_ativos_cadastro.csv carregado para a tabela 2024_fev_militares_ativos_cadastro no BigQuery.
Arquivo 2024_mar_militares_ativos_cadastro.csv não encontrado.


In [34]:
# MILITRES ATIVOS REMUNERAÇÃO<< 
for ano in anos:
    for mes in meses:
        # Nome do arquivo 
        nome_arquivo = f"{ano}_{mes}_militares_ativos_remuneracao.csv"
        caminho_arquivo = os.path.join(diretorio_remuneracao, nome_arquivo)

        # Verificar se o arquivo existe
        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            df = df.drop(df.index[-1])
            df['ANO'] = df['ANO'].astype(int)
            df = df.rename(columns={
                                    'REMUNERAÇÃO BÁSICA BRUTA (R$)': 'REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                                    'REMUNERAÇÃO BÁSICA BRUTA (U$)': 'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                                    'ABATE-TETO (R$)': 'ABATE_TETO_EM_REAIS',
                                    'ABATE-TETO (U$)': 'ABATE_TETO_EM_DOLARES',
                                    'GRATIFICAÇÃO NATALINA (R$)': 'GRATIFICACAO_NATALINA_EM_REAIS',
                                    'GRATIFICAÇÃO NATALINA (U$)': 'GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'FÉRIAS (R$)': 'FERIAS_EM_REAIS',
                                    'FÉRIAS (U$)': 'FERIAS_EM_DOLARES',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                                    'IRRF (R$)': 'IRRF_EM_REAIS',
                                    'IRRF (U$)': 'IRRF_EM_DOLARES',
                                    'PSS/RPGS (R$)': 'PSS_RPGS_EM_REAIS',
                                    'PSS/RPGS (U$)': 'PSS_RPGS_EM_DOLARES',
                                    'DEMAIS DEDUÇÕES (R$)': 'DEMAIS_DEDUCOES_EM_REAIS',
                                    'DEMAIS DEDUÇÕES (U$)': 'DEMAIS_DEDUCOES_EM_DOLARES',
                                    'PENSÃO MILITAR (R$)': 'PENSAO_MILITAR_EM_REAIS',
                                    'PENSÃO MILITAR (U$)': 'PENSAO_MILITAR_EM_DOLARES',
                                    'FUNDO DE SAÚDE (R$)': 'FUNDO_DE_SAUDE_EM_REAIS',
                                    'FUNDO DE SAÚDE (U$)': 'FUNDO_DE_SAUDE_EM_DOLARES',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                                    })
            
            #Mapeando as colunas que precisam ser convertidas para float
            colunas_float = ['REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                            'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                            'ABATE_TETO_EM_REAIS',
                            'ABATE_TETO_EM_DOLARES', 
                            'GRATIFICACAO_NATALINA_EM_REAIS',
                            'GRATIFICACAO_NATALINA_EM_DOLARES',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 
                            'FERIAS_EM_REAIS',
                            'FERIAS_EM_DOLARES', 
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                            'IRRF_EM_REAIS',
                            'IRRF_EM_DOLARES',
                            'PSS_RPGS_EM_REAIS',
                            'PSS_RPGS_EM_DOLARES',
                            'DEMAIS_DEDUCOES_EM_REAIS',
                            'DEMAIS_DEDUCOES_EM_DOLARES',
                            'PENSAO_MILITAR_EM_REAIS',
                            'PENSAO_MILITAR_EM_DOLARES',
                            'FUNDO_DE_SAUDE_EM_REAIS',
                            'FUNDO_DE_SAUDE_EM_DOLARES',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                            ]
            #Converter as colunas para float
            for coluna in colunas_float:
                df[coluna] = df[coluna].str.replace(',', '.').astype(float)
                        
            #Carregar o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_militares_ativos_remuneracao"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema =  Schema= ([bigquery.SchemaField('ANO', 'INTEGER', description=''),
                                bigquery.SchemaField('MES', 'FLOAT', description=''),
                                bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'FLOAT', description=''),
                                bigquery.SchemaField('CPF', 'STRING', description=''),
                                bigquery.SchemaField('NOME', 'STRING', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES', 'FLOAT', description='')]
                                )
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_7144\187781133.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_jan_militares_ativos_remuneracao.csv carregado para a tabela 2024_jan_militares_ativos_remuneracao no BigQuery.


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_7144\187781133.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_fev_militares_ativos_remuneracao.csv carregado para a tabela 2024_fev_militares_ativos_remuneracao no BigQuery.
Arquivo 2024_mar_militares_ativos_remuneracao.csv não encontrado.


In [23]:
##MILITARES RESERVA REFORMA CADASTRO
for ano in anos:
    for mes_num, mes in enumerate(meses, start=3):
        nome_arquivo = f"{ano}_{mes}_militares_reserva_reforma_cadastro.csv"
        caminho_arquivo = os.path.join(diretorio_cadastro, nome_arquivo)

        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            
            # Criar colunas de ano e mês de acordo com o nome do arquivo
            df['ANO'] = ano
            df['MES'] = mes_num
            # Carregando o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_militares_reserva_reforma_cadastro"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema = ([bigquery.SchemaField('ANO', 'INTEGER', description=''),
                            bigquery.SchemaField('MES', 'INTEGER', description=''),
                            bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'INTEGER', description=''),
                            bigquery.SchemaField('NOME', 'STRING', description=''),
                            bigquery.SchemaField('CPF', 'STRING', description=''),
                            bigquery.SchemaField('MATRICULA', 'STRING', description=''),
                            bigquery.SchemaField('COD_TIPO_APOSENTADORIA', 'INTEGER', description=''),
                            bigquery.SchemaField('TIPO_APOSENTADORIA', 'STRING', description=''),
                            bigquery.SchemaField('DATA_APOSENTADORIA', 'STRING', description=''),
                            bigquery.SchemaField('DESCRICAO_CARGO', 'STRING', description=''),
                            bigquery.SchemaField('COD_UORG_LOTACAO', 'INTEGER', description=''),
                            bigquery.SchemaField('UORG_LOTACAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_ORG_LOTACAO', 'INTEGER', description=''),
                            bigquery.SchemaField('ORG_LOTACAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_ORGSUP_LOTACAO', 'INTEGER', description=''),
                            bigquery.SchemaField('ORGSUP_LOTACAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_TIPO_VINCULO', 'INTEGER', description=''),
                            bigquery.SchemaField('TIPO_VINCULO', 'STRING', description=''),
                            bigquery.SchemaField('SITUACAO_VINCULO', 'STRING', description=''),
                            bigquery.SchemaField('REGIME_JURIDICO', 'STRING', description=''),
                            bigquery.SchemaField('JORNADA_DE_TRABALHO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_INGRESSO_CARGOFUNCAO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_NOMEACAO_CARGOFUNCAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DATA_INGRESSO_ORGAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DOCUMENTO_INGRESSO_SERVICOPUBLICO', 'INTEGER', description=''),
                            bigquery.SchemaField('DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO', 'FLOAT', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_CARGOFUNCAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_ORGAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_SERVICOPUBLICO', 'STRING', description='')
                            ])

        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")


Arquivo 2024_mar_militares_reserva_reforma_cadastro.csv carregado para a tabela 2024_mar_militares_reserva_reforma_cadastro no BigQuery.
Arquivo 2024_abr_militares_reserva_reforma_cadastro.csv carregado para a tabela 2024_abr_militares_reserva_reforma_cadastro no BigQuery.
Arquivo 2024_mai_militares_reserva_reforma_cadastro.csv carregado para a tabela 2024_mai_militares_reserva_reforma_cadastro no BigQuery.


In [24]:
diretorio_remuneracao = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Atualização Dados novos\\CGU\\MILITARES\\RESERVA_REFORMA_REMUNERACAO'

In [25]:
os.chdir(diretorio_remuneracao)
os.listdir(diretorio_remuneracao)

['2023_mar_militares_reserva_reforma_remuneracao.csv',
 '2023_fev_militares_reserva_reforma_remuneracao.csv',
 '2023_jan_militares_reserva_reforma_remuneracao.csv',
 '2022_dez_militares_reserva_reforma_remuneracao.csv',
 '2022_nov_militares_reserva_reforma_remuneracao.csv',
 '2022_out_militares_reserva_reforma_remuneracao.csv',
 '2022_set_militares_reserva_reforma_remuneracao.csv',
 '2022_ago_militares_reserva_reforma_remuneracao.csv',
 '2022_jul_militares_reserva_reforma_remuneracao.csv',
 '2022_jun_militares_reserva_reforma_remuneracao.csv',
 '2022_mai_militares_reserva_reforma_remuneracao.csv',
 '2022_abr_militares_reserva_reforma_remuneracao.csv',
 '2022_mar_militares_reserva_reforma_remuneracao.csv',
 '2022_fev_militares_reserva_reforma_remuneracao.csv',
 '2022_jan_militares_reserva_reforma_remuneracao.csv',
 '2021_dez_militares_reserva_reforma_remuneracao.csv',
 '2021_nov_militares_reserva_reforma_remuneracao.csv',
 '2021_out_militares_reserva_reforma_remuneracao.csv',
 '2021_set

In [29]:
# MILITRES RESERVA-REFORMA REMUNERAÇÃO<< 
for ano in anos:
    for mes in meses:
        # Nome do arquivo 
        nome_arquivo = f"{ano}_{mes}_militares_reserva_reforma_remuneracao.csv"
        caminho_arquivo = os.path.join(diretorio_remuneracao, nome_arquivo)

        # Verificar se o arquivo existe
        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            df = df.drop(df.index[-1])
            df['ANO'] = df['ANO'].astype(int)
            df = df.rename(columns={
                                    'REMUNERAÇÃO BÁSICA BRUTA (R$)': 'REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                                    'REMUNERAÇÃO BÁSICA BRUTA (U$)': 'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                                    'ABATE-TETO (R$)': 'ABATE_TETO_EM_REAIS',
                                    'ABATE-TETO (U$)': 'ABATE_TETO_EM_DOLARES',
                                    'GRATIFICAÇÃO NATALINA (R$)': 'GRATIFICACAO_NATALINA_EM_REAIS',
                                    'GRATIFICAÇÃO NATALINA (U$)': 'GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'FÉRIAS (R$)': 'FERIAS_EM_REAIS',
                                    'FÉRIAS (U$)': 'FERIAS_EM_DOLARES',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                                    'IRRF (R$)': 'IRRF_EM_REAIS',
                                    'IRRF (U$)': 'IRRF_EM_DOLARES',
                                    'PSS/RPGS (R$)': 'PSS_RPGS_EM_REAIS',
                                    'PSS/RPGS (U$)': 'PSS_RPGS_EM_DOLARES',
                                    'DEMAIS DEDUÇÕES (R$)': 'DEMAIS_DEDUCOES_EM_REAIS',
                                    'DEMAIS DEDUÇÕES (U$)': 'DEMAIS_DEDUCOES_EM_DOLARES',
                                    'PENSÃO MILITAR (R$)': 'PENSAO_MILITAR_EM_REAIS',
                                    'PENSÃO MILITAR (U$)': 'PENSAO_MILITAR_EM_DOLARES',
                                    'FUNDO DE SAÚDE (R$)': 'FUNDO_DE_SAUDE_EM_REAIS',
                                    'FUNDO DE SAÚDE (U$)': 'FUNDO_DE_SAUDE_EM_DOLARES',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                                    })
            
            #Mapeando as colunas que precisam ser convertidas para float
            colunas_float = ['REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                            'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                            'ABATE_TETO_EM_REAIS',
                            'ABATE_TETO_EM_DOLARES', 
                            'GRATIFICACAO_NATALINA_EM_REAIS',
                            'GRATIFICACAO_NATALINA_EM_DOLARES',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 
                            'FERIAS_EM_REAIS',
                            'FERIAS_EM_DOLARES', 
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                            'IRRF_EM_REAIS',
                            'IRRF_EM_DOLARES',
                            'PSS_RPGS_EM_REAIS',
                            'PSS_RPGS_EM_DOLARES',
                            'DEMAIS_DEDUCOES_EM_REAIS',
                            'DEMAIS_DEDUCOES_EM_DOLARES',
                            'PENSAO_MILITAR_EM_REAIS',
                            'PENSAO_MILITAR_EM_DOLARES',
                            'FUNDO_DE_SAUDE_EM_REAIS',
                            'FUNDO_DE_SAUDE_EM_DOLARES',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                            ]
            #Converter as colunas para float
            for coluna in colunas_float:
                df[coluna] = df[coluna].str.replace(',', '.').astype(float)
                        
            #Carregar o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_militares_reserva_reforma_remuneracao"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema =  Schema= ([bigquery.SchemaField('ANO', 'INTEGER', description=''),
                                bigquery.SchemaField('MES', 'FLOAT', description=''),
                                bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'FLOAT', description=''),
                                bigquery.SchemaField('CPF', 'STRING', description=''),
                                bigquery.SchemaField('NOME', 'STRING', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES', 'FLOAT', description='')]
                                )
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_11936\3504373915.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_mar_militares_reserva_reforma_remuneracao.csv carregado para a tabela 2024_mar_militares_reserva_reforma_remuneracao no BigQuery.


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_11936\3504373915.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_abr_militares_reserva_reforma_remuneracao.csv carregado para a tabela 2024_abr_militares_reserva_reforma_remuneracao no BigQuery.


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_11936\3504373915.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_mai_militares_reserva_reforma_remuneracao.csv carregado para a tabela 2024_mai_militares_reserva_reforma_remuneracao no BigQuery.


In [13]:
# MILITARES PENSIONISTAS CADASTRO

for ano in anos:
    for mes_num, mes in enumerate(meses, start=#): Iniciar conforme o ano que vai subir no GBQ
        nome_arquivo = f"{ano}_{mes}_militares_pensionistas_cadastro.csv"
        caminho_arquivo = os.path.join(diretorio_cadastro, nome_arquivo)

        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            
            # Criar colunas de ano e mês de acordo com o nome do arquivo
            df['ANO'] = ano
            df['MES'] = mes_num

            # Carregando o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_militares_pensionistas_cadastro"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema = ([
                            bigquery.SchemaField('ANO', 'INTEGER', description=''),
                            bigquery.SchemaField('MES', 'INTEGER', description=''),
                            bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'INTEGER', description=''),
                            bigquery.SchemaField('NOME', 'STRING', description=''),
                            bigquery.SchemaField('CPF', 'STRING', description=''),
                            bigquery.SchemaField('MATRICULA', 'STRING', description=''),
                            bigquery.SchemaField('CPF_REPRESENTANTE_LEGAL', 'STRING', description=''),
                            bigquery.SchemaField('NOME_REPRESENTANTE_LEGAL', 'STRING', description=''),
                            bigquery.SchemaField('CPF_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('NOME_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_TIPO_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('TIPO_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_INICIO_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DESCRICAO_CARGO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_UORG_LOTACAO_INSTITUIDOR_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('UORG_LOTACAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_ORG_LOTACAO_INSTITUIDOR_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('ORG_LOTACAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_ORGSUP_LOTACAO_INSTITUIDOR_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('ORGSUP_LOTACAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_TIPO_VINCULO', 'INTEGER', description=''),
                            bigquery.SchemaField('TIPO_VINCULO', 'STRING', description=''),
                            bigquery.SchemaField('SITUACAO_VINCULO', 'STRING', description=''),
                            bigquery.SchemaField('REGIME_JURIDICO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('JORNADA_DE_TRABALHO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_INGRESSO_CARGOFUNCAO_INSTITUIDOR_PENSAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DATA_NOMEACAO_CARGOFUNCAO_INSTITUIDOR_PENSAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DATA_INGRESSO_ORGAO_INSTITUIDOR_PENSAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DOCUMENTO_INGRESSO_SERVICOPUBLICO_INSTITUIDOR_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO_INSTITUIDOR_PENSAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_CARGOFUNCAO_INSTITUIDOR_PENSAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_ORGAO_INSTITUIDOR_PENSAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_SERVICOPUBLICO_INSTITUIDOR_PENSAO', 'FLOAT', description='')
                            ])
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

Arquivo 2024_mar_militares_pensionistas_cadastro.csv carregado para a tabela 2024_mar_militares_pensionistas_cadastro no BigQuery.
Arquivo 2024_abr_militares_pensionistas_cadastro.csv carregado para a tabela 2024_abr_militares_pensionistas_cadastro no BigQuery.


In [18]:
# MILITARES PENSIONISTAS REMUNERAÇÃO
for ano in anos:
    for mes in meses:
        # Nome do arquivo 
        nome_arquivo = f"{ano}_{mes}_militares_pensionistas_remuneracao.csv"
        caminho_arquivo = os.path.join(diretorio_remuneracao, nome_arquivo)

        # Verificar se o arquivo existe
        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            df = df.drop(df.index[-1])
            df['ANO'] = df['ANO'].astype(int)
            df = df.rename(columns={
                                    'REMUNERAÇÃO BÁSICA BRUTA (R$)': 'REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                                    'REMUNERAÇÃO BÁSICA BRUTA (U$)': 'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                                    'ABATE-TETO (R$)': 'ABATE_TETO_EM_REAIS',
                                    'ABATE-TETO (U$)': 'ABATE_TETO_EM_DOLARES',
                                    'GRATIFICAÇÃO NATALINA (R$)': 'GRATIFICACAO_NATALINA_EM_REAIS',
                                    'GRATIFICAÇÃO NATALINA (U$)': 'GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'FÉRIAS (R$)': 'FERIAS_EM_REAIS',
                                    'FÉRIAS (U$)': 'FERIAS_EM_DOLARES',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                                    'IRRF (R$)': 'IRRF_EM_REAIS',
                                    'IRRF (U$)': 'IRRF_EM_DOLARES',
                                    'PSS/RPGS (R$)': 'PSS_RPGS_EM_REAIS',
                                    'PSS/RPGS (U$)': 'PSS_RPGS_EM_DOLARES',
                                    'DEMAIS DEDUÇÕES (R$)': 'DEMAIS_DEDUCOES_EM_REAIS',
                                    'DEMAIS DEDUÇÕES (U$)': 'DEMAIS_DEDUCOES_EM_DOLARES',
                                    'PENSÃO MILITAR (R$)': 'PENSAO_MILITAR_EM_REAIS',
                                    'PENSÃO MILITAR (U$)': 'PENSAO_MILITAR_EM_DOLARES',
                                    'FUNDO DE SAÚDE (R$)': 'FUNDO_DE_SAUDE_EM_REAIS',
                                    'FUNDO DE SAÚDE (U$)': 'FUNDO_DE_SAUDE_EM_DOLARES',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                                    })
            
            #Mapeando as colunas que precisam ser convertidas para float
            colunas_float = ['REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                            'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                            'ABATE_TETO_EM_REAIS',
                            'ABATE_TETO_EM_DOLARES', 
                            'GRATIFICACAO_NATALINA_EM_REAIS',
                            'GRATIFICACAO_NATALINA_EM_DOLARES',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 
                            'FERIAS_EM_REAIS',
                            'FERIAS_EM_DOLARES', 
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                            'IRRF_EM_REAIS',
                            'IRRF_EM_DOLARES',
                            'PSS_RPGS_EM_REAIS',
                            'PSS_RPGS_EM_DOLARES',
                            'DEMAIS_DEDUCOES_EM_REAIS',
                            'DEMAIS_DEDUCOES_EM_DOLARES',
                            'PENSAO_MILITAR_EM_REAIS',
                            'PENSAO_MILITAR_EM_DOLARES',
                            'FUNDO_DE_SAUDE_EM_REAIS',
                            'FUNDO_DE_SAUDE_EM_DOLARES',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                            ]
            #Converter as colunas para float
            for coluna in colunas_float:
                df[coluna] = df[coluna].str.replace(',', '.').astype(float)
                        
            #Carregar o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_militares_pensionistas_remuneracao"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema =  Schema= ([bigquery.SchemaField('ANO', 'INTEGER', description=''),
                                bigquery.SchemaField('MES', 'FLOAT', description=''),
                                bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'FLOAT', description=''),
                                bigquery.SchemaField('CPF', 'STRING', description=''),
                                bigquery.SchemaField('NOME', 'STRING', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES', 'FLOAT', description='')]
                                )
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_13252\3877133208.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_mar_militares_pensionistas_remuneracao.csv carregado para a tabela 2024_mar_militares_pensionistas_remuneracao no BigQuery.


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_13252\3877133208.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_abr_militares_pensionistas_remuneracao.csv carregado para a tabela 2024_abr_militares_pensionistas_remuneracao no BigQuery.


### SIAPE

In [94]:
# SIAPE APOSENTADOS CADASTRO
for ano in anos:
    for mes_num, mes in enumerate(meses, start=1):
        nome_arquivo = f"{ano}_{mes}_siape_aposentados_cadastro.csv"
        caminho_arquivo = os.path.join(diretorio_cadastro, nome_arquivo)

        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            
            # Criar colunas de ano e mês de acordo com o nome do arquivo
            df = df.drop(df.index[df['NOME']=='Sigiloso'])
            df['ANO'] = ano
            df['MES'] = mes_num

            # Carregando o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_siape_aposentados_cadastro"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema = ([
                            bigquery.SchemaField('ANO', 'INTEGER', description=''),
                            bigquery.SchemaField('MES', 'INTEGER', description=''),
                            bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'INTEGER', description=''),
                            bigquery.SchemaField('NOME', 'STRING', description=''),
                            bigquery.SchemaField('CPF', 'STRING', description=''),
                            bigquery.SchemaField('MATRICULA', 'STRING', description=''),
                            bigquery.SchemaField('COD_TIPO_APOSENTADORIA', 'INTEGER', description=''),
                            bigquery.SchemaField('TIPO_APOSENTADORIA', 'STRING', description=''),
                            bigquery.SchemaField('DATA_APOSENTADORIA', 'STRING', description=''),
                            bigquery.SchemaField('DESCRICAO_CARGO', 'STRING', description=''),
                            bigquery.SchemaField('COD_UORG_LOTACAO', 'INTEGER', description=''),
                            bigquery.SchemaField('UORG_LOTACAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_ORG_LOTACAO', 'INTEGER', description=''),
                            bigquery.SchemaField('ORG_LOTACAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_ORGSUP_LOTACAO', 'INTEGER', description=''),
                            bigquery.SchemaField('ORGSUP_LOTACAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_TIPO_VINCULO', 'INTEGER', description=''),
                            bigquery.SchemaField('TIPO_VINCULO', 'STRING', description=''),
                            bigquery.SchemaField('SITUACAO_VINCULO', 'STRING', description=''),
                            bigquery.SchemaField('REGIME_JURIDICO', 'STRING', description=''),
                            bigquery.SchemaField('JORNADA_DE_TRABALHO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_INGRESSO_CARGOFUNCAO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_NOMEACAO_CARGOFUNCAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DATA_INGRESSO_ORGAO', 'STRING', description=''),
                            bigquery.SchemaField('DOCUMENTO_INGRESSO_SERVICOPUBLICO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO', 'STRING', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_CARGOFUNCAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_ORGAO', 'STRING', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_SERVICOPUBLICO', 'STRING', description='')
                            ])
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

Arquivo 2024_jan_siape_aposentados_cadastro.csv carregado para a tabela 2024_jan_siape_aposentados_cadastro no BigQuery.
Arquivo 2024_fev_siape_aposentados_cadastro.csv carregado para a tabela 2024_fev_siape_aposentados_cadastro no BigQuery.
Arquivo 2024_mar_siape_aposentados_cadastro.csv carregado para a tabela 2024_mar_siape_aposentados_cadastro no BigQuery.


In [109]:
# SIAPE APOSENTADOS REMUNERAÇÃO<< 
for ano in anos:
    for mes in meses:
        # Nome do arquivo 
        nome_arquivo = f"{ano}_{mes}_siape_aposentados_remuneracao.csv"
        caminho_arquivo = os.path.join(diretorio_remuneracao, nome_arquivo)

        # Verificar se o arquivo existe
        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            df = df.drop(df.index[-1])
            df['ANO'] = df['ANO'].astype(int)
            df = df.rename(columns={
                                    'REMUNERAÇÃO BÁSICA BRUTA (R$)': 'REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                                    'REMUNERAÇÃO BÁSICA BRUTA (U$)': 'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                                    'ABATE-TETO (R$)': 'ABATE_TETO_EM_REAIS',
                                    'ABATE-TETO (U$)': 'ABATE_TETO_EM_DOLARES',
                                    'GRATIFICAÇÃO NATALINA (R$)': 'GRATIFICACAO_NATALINA_EM_REAIS',
                                    'GRATIFICAÇÃO NATALINA (U$)': 'GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'FÉRIAS (R$)': 'FERIAS_EM_REAIS',
                                    'FÉRIAS (U$)': 'FERIAS_EM_DOLARES',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                                    'IRRF (R$)': 'IRRF_EM_REAIS',
                                    'IRRF (U$)': 'IRRF_EM_DOLARES',
                                    'PSS/RPGS (R$)': 'PSS_RPGS_EM_REAIS',
                                    'PSS/RPGS (U$)': 'PSS_RPGS_EM_DOLARES',
                                    'DEMAIS DEDUÇÕES (R$)': 'DEMAIS_DEDUCOES_EM_REAIS',
                                    'DEMAIS DEDUÇÕES (U$)': 'DEMAIS_DEDUCOES_EM_DOLARES',
                                    'PENSÃO MILITAR (R$)': 'PENSAO_MILITAR_EM_REAIS',
                                    'PENSÃO MILITAR (U$)': 'PENSAO_MILITAR_EM_DOLARES',
                                    'FUNDO DE SAÚDE (R$)': 'FUNDO_DE_SAUDE_EM_REAIS',
                                    'FUNDO DE SAÚDE (U$)': 'FUNDO_DE_SAUDE_EM_DOLARES',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                                    })
            
            #Mapeando as colunas que precisam ser convertidas para float
            colunas_float = ['REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                            'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                            'ABATE_TETO_EM_REAIS',
                            'ABATE_TETO_EM_DOLARES', 
                            'GRATIFICACAO_NATALINA_EM_REAIS',
                            'GRATIFICACAO_NATALINA_EM_DOLARES',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 
                            'FERIAS_EM_REAIS',
                            'FERIAS_EM_DOLARES', 
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                            'IRRF_EM_REAIS',
                            'IRRF_EM_DOLARES',
                            'PSS_RPGS_EM_REAIS',
                            'PSS_RPGS_EM_DOLARES',
                            'DEMAIS_DEDUCOES_EM_REAIS',
                            'DEMAIS_DEDUCOES_EM_DOLARES',
                            'PENSAO_MILITAR_EM_REAIS',
                            'PENSAO_MILITAR_EM_DOLARES',
                            'FUNDO_DE_SAUDE_EM_REAIS',
                            'FUNDO_DE_SAUDE_EM_DOLARES',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                            ]
            #Converter as colunas para float
            for coluna in colunas_float:
                df[coluna] = df[coluna].str.replace(',', '.').astype(float)
                        
            #Carregar o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_siape_aposentados_remuneracao"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema =  Schema= ([bigquery.SchemaField('ANO', 'INTEGER', description=''),
                                bigquery.SchemaField('MES', 'FLOAT', description=''),
                                bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'FLOAT', description=''),
                                bigquery.SchemaField('CPF', 'STRING', description=''),
                                bigquery.SchemaField('NOME', 'STRING', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES', 'FLOAT', description='')]
                                )
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_10172\2578436462.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_jan_siape_aposentados_remuneracao.csv carregado para a tabela 2024_jan_siape_aposentados_remuneracao no BigQuery.


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_10172\2578436462.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_fev_siape_aposentados_remuneracao.csv carregado para a tabela 2024_fev_siape_aposentados_remuneracao no BigQuery.


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_10172\2578436462.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_mar_siape_aposentados_remuneracao.csv carregado para a tabela 2024_mar_siape_aposentados_remuneracao no BigQuery.


In [135]:
# SIAPE ATIVOS CADASTRO
for ano in anos:
    for mes_num, mes in enumerate(meses, start=1):
        nome_arquivo = f"{ano}_{mes}_siape_servidores_cadastro.csv"
        caminho_arquivo = os.path.join(diretorio_cadastro, nome_arquivo)

        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            
            # Criar colunas de ano e mês de acordo com o nome do arquivo
            df = df.drop(df.index[df['NOME']=='Sigiloso'])
            df['ANO'] = ano
            df['MES'] = mes_num

            # Carregando o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_siape_servidores_cadastro"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema = ([
                    bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'INTEGER', description=''),
                    bigquery.SchemaField('NOME', 'STRING', description=''),
                    bigquery.SchemaField('CPF', 'STRING', description=''),
                    bigquery.SchemaField('MATRICULA', 'STRING', description=''),
                    bigquery.SchemaField('DESCRICAO_CARGO', 'STRING', description=''),
                    bigquery.SchemaField('CLASSE_CARGO', 'STRING', description=''),
                    bigquery.SchemaField('REFERENCIA_CARGO', 'FLOAT', description=''),
                    bigquery.SchemaField('PADRAO_CARGO', 'STRING', description=''),
                    bigquery.SchemaField('NIVEL_CARGO', 'FLOAT', description=''),
                    bigquery.SchemaField('SIGLA_FUNCAO', 'STRING', description=''),
                    bigquery.SchemaField('NIVEL_FUNCAO', 'FLOAT', description=''),
                    bigquery.SchemaField('FUNCAO', 'STRING', description=''),
                    bigquery.SchemaField('CODIGO_ATIVIDADE', 'INTEGER', description=''),
                    bigquery.SchemaField('ATIVIDADE', 'STRING', description=''),
                    bigquery.SchemaField('OPCAO_PARCIAL', 'STRING', description=''),
                    bigquery.SchemaField('COD_UORG_LOTACAO', 'INTEGER', description=''),
                    bigquery.SchemaField('UORG_LOTACAO', 'STRING', description=''),
                    bigquery.SchemaField('COD_ORG_LOTACAO', 'INTEGER', description=''),
                    bigquery.SchemaField('ORG_LOTACAO', 'STRING', description=''),
                    bigquery.SchemaField('COD_ORGSUP_LOTACAO', 'INTEGER', description=''),
                    bigquery.SchemaField('ORGSUP_LOTACAO', 'STRING', description=''),
                    bigquery.SchemaField('COD_UORG_EXERCICIO', 'INTEGER', description=''),
                    bigquery.SchemaField('UORG_EXERCICIO', 'STRING', description=''),
                    bigquery.SchemaField('COD_ORG_EXERCICIO', 'INTEGER', description=''),
                    bigquery.SchemaField('ORG_EXERCICIO', 'STRING', description=''),
                    bigquery.SchemaField('COD_ORGSUP_EXERCICIO', 'INTEGER', description=''),
                    bigquery.SchemaField('ORGSUP_EXERCICIO', 'STRING', description=''),
                    bigquery.SchemaField('COD_TIPO_VINCULO', 'INTEGER', description=''),
                    bigquery.SchemaField('TIPO_VINCULO', 'STRING', description=''),
                    bigquery.SchemaField('SITUACAO_VINCULO', 'STRING', description=''),
                    bigquery.SchemaField('DATA_INICIO_AFASTAMENTO', 'FLOAT', description=''),
                    bigquery.SchemaField('DATA_TERMINO_AFASTAMENTO', 'FLOAT', description=''),
                    bigquery.SchemaField('REGIME_JURIDICO', 'STRING', description=''),
                    bigquery.SchemaField('JORNADA_DE_TRABALHO', 'STRING', description=''),
                    bigquery.SchemaField('DATA_INGRESSO_CARGOFUNCAO', 'STRING', description=''),
                    bigquery.SchemaField('DATA_NOMEACAO_CARGOFUNCAO', 'FLOAT', description=''),
                    bigquery.SchemaField('DATA_INGRESSO_ORGAO', 'STRING', description=''),
                    bigquery.SchemaField('DOCUMENTO_INGRESSO_SERVICOPUBLICO', 'STRING', description=''),
                    bigquery.SchemaField('DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO', 'STRING', description=''),
                    bigquery.SchemaField('DIPLOMA_INGRESSO_CARGOFUNCAO', 'FLOAT', description=''),
                    bigquery.SchemaField('DIPLOMA_INGRESSO_ORGAO', 'STRING', description=''),
                    bigquery.SchemaField('DIPLOMA_INGRESSO_SERVICOPUBLICO', 'STRING', description=''),
                    bigquery.SchemaField('UF_EXERCICIO', 'INTEGER', description=''),
                    bigquery.SchemaField('ANO', 'INTEGER', description=''),
                    bigquery.SchemaField('MES', 'INTEGER', description='')

                    ])
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_10172\948976412.py:9: DtypeWarning: Columns (2,5,7,9,34,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_jan_siape_servidores_cadastro.csv carregado para a tabela 2024_jan_siape_servidores_cadastro no BigQuery.


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_10172\948976412.py:9: DtypeWarning: Columns (2,5,7,9,34,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_fev_siape_servidores_cadastro.csv carregado para a tabela 2024_fev_siape_servidores_cadastro no BigQuery.


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_10172\948976412.py:9: DtypeWarning: Columns (2,5,7,9,34,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_mar_siape_servidores_cadastro.csv carregado para a tabela 2024_mar_siape_servidores_cadastro no BigQuery.


In [142]:
# SIAPE SERVIDORES ATIVOS REMUNERAÇÃO
for ano in anos:
    for mes in meses:
        # Nome do arquivo 
        nome_arquivo = f"{ano}_{mes}_siape_servidores_remuneracao.csv"
        caminho_arquivo = os.path.join(diretorio_remuneracao, nome_arquivo)

        # Verificar se o arquivo existe
        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            df = df.drop(df.index[-1])
            df['ANO'] = df['ANO'].astype(int)
            df = df.rename(columns={
                                    'REMUNERAÇÃO BÁSICA BRUTA (R$)': 'REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                                    'REMUNERAÇÃO BÁSICA BRUTA (U$)': 'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                                    'ABATE-TETO (R$)': 'ABATE_TETO_EM_REAIS',
                                    'ABATE-TETO (U$)': 'ABATE_TETO_EM_DOLARES',
                                    'GRATIFICAÇÃO NATALINA (R$)': 'GRATIFICACAO_NATALINA_EM_REAIS',
                                    'GRATIFICAÇÃO NATALINA (U$)': 'GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'FÉRIAS (R$)': 'FERIAS_EM_REAIS',
                                    'FÉRIAS (U$)': 'FERIAS_EM_DOLARES',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                                    'IRRF (R$)': 'IRRF_EM_REAIS',
                                    'IRRF (U$)': 'IRRF_EM_DOLARES',
                                    'PSS/RPGS (R$)': 'PSS_RPGS_EM_REAIS',
                                    'PSS/RPGS (U$)': 'PSS_RPGS_EM_DOLARES',
                                    'DEMAIS DEDUÇÕES (R$)': 'DEMAIS_DEDUCOES_EM_REAIS',
                                    'DEMAIS DEDUÇÕES (U$)': 'DEMAIS_DEDUCOES_EM_DOLARES',
                                    'PENSÃO MILITAR (R$)': 'PENSAO_MILITAR_EM_REAIS',
                                    'PENSÃO MILITAR (U$)': 'PENSAO_MILITAR_EM_DOLARES',
                                    'FUNDO DE SAÚDE (R$)': 'FUNDO_DE_SAUDE_EM_REAIS',
                                    'FUNDO DE SAÚDE (U$)': 'FUNDO_DE_SAUDE_EM_DOLARES',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'

                                    })
            #Mapeando as colunas que precisam ser convertidas para float
            colunas_float = ['REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                            'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                            'ABATE_TETO_EM_REAIS',
                            'ABATE_TETO_EM_DOLARES', 
                            'GRATIFICACAO_NATALINA_EM_REAIS',
                            'GRATIFICACAO_NATALINA_EM_DOLARES',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 
                            'FERIAS_EM_REAIS',
                            'FERIAS_EM_DOLARES', 
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                            'IRRF_EM_REAIS',
                            'IRRF_EM_DOLARES',
                            'PSS_RPGS_EM_REAIS',
                            'PSS_RPGS_EM_DOLARES',
                            'DEMAIS_DEDUCOES_EM_REAIS',
                            'DEMAIS_DEDUCOES_EM_DOLARES',
                            'PENSAO_MILITAR_EM_REAIS',
                            'PENSAO_MILITAR_EM_DOLARES',
                            'FUNDO_DE_SAUDE_EM_REAIS',
                            'FUNDO_DE_SAUDE_EM_DOLARES',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                            ]
            #Converter as colunas para float
            for coluna in colunas_float:
                df[coluna] = df[coluna].str.replace(',', '.').astype(float)
                        
            #Carregar o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_siape_servidores_remuneracao"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema =  Schema= ([bigquery.SchemaField('ANO', 'INTEGER', description=''),
                                bigquery.SchemaField('MES', 'FLOAT', description=''),
                                bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'FLOAT', description=''),
                                bigquery.SchemaField('CPF', 'STRING', description=''),
                                bigquery.SchemaField('NOME', 'STRING', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES', 'FLOAT', description='')]
                                )
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_10172\2323118395.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_jan_siape_servidores_remuneracao.csv carregado para a tabela 2024_jan_siape_servidores_remuneracao no BigQuery.


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_10172\2323118395.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_fev_siape_servidores_remuneracao.csv carregado para a tabela 2024_fev_siape_servidores_remuneracao no BigQuery.


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_10172\2323118395.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_mar_siape_servidores_remuneracao.csv carregado para a tabela 2024_mar_siape_servidores_remuneracao no BigQuery.


In [127]:
# SIAPE PENSIONISTAS CADASTRO
for ano in anos:
    for mes_num, mes in enumerate(meses, start=1):
        nome_arquivo = f"{ano}_{mes}_siape_pensionistas_cadastro.csv"
        caminho_arquivo = os.path.join(diretorio_cadastro, nome_arquivo)

        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            
            # Criar colunas de ano e mês de acordo com o nome do arquivo
            df = df.drop(df.index[df['NOME']=='Sigiloso'])
            df['ANO'] = ano
            df['MES'] = mes_num

            # Carregando o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_siape_pensionista_cadastro"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema = ([
                            bigquery.SchemaField('ANO', 'INTEGER', description=''),
                            bigquery.SchemaField('MES', 'INTEGER', description=''),
                            bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'INTEGER', description=''),
                            bigquery.SchemaField('NOME', 'STRING', description=''),
                            bigquery.SchemaField('CPF', 'STRING', description=''),
                            bigquery.SchemaField('MATRICULA', 'STRING', description=''),
                            bigquery.SchemaField('CPF_REPRESENTANTE_LEGAL', 'STRING', description=''),
                            bigquery.SchemaField('NOME_REPRESENTANTE_LEGAL', 'STRING', description=''),
                            bigquery.SchemaField('CPF_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('NOME_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_TIPO_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('TIPO_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_INICIO_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DESCRICAO_CARGO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_UORG_LOTACAO_INSTITUIDOR_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('UORG_LOTACAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_ORG_LOTACAO_INSTITUIDOR_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('ORG_LOTACAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_ORGSUP_LOTACAO_INSTITUIDOR_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('ORGSUP_LOTACAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_TIPO_VINCULO', 'INTEGER', description=''),
                            bigquery.SchemaField('TIPO_VINCULO', 'STRING', description=''),
                            bigquery.SchemaField('SITUACAO_VINCULO', 'STRING', description=''),
                            bigquery.SchemaField('REGIME_JURIDICO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('JORNADA_DE_TRABALHO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_INGRESSO_CARGOFUNCAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_NOMEACAO_CARGOFUNCAO_INSTITUIDOR_PENSAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DATA_INGRESSO_ORGAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DOCUMENTO_INGRESSO_SERVICOPUBLICO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_CARGOFUNCAO_INSTITUIDOR_PENSAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_ORGAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_SERVICOPUBLICO_INSTITUIDOR_PENSAO', 'STRING', description='')
                            ])
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

Arquivo 2024_jan_siape_pensionistas_cadastro.csv carregado para a tabela 2024_jan_siape_pensionista_cadastro no BigQuery.
Arquivo 2024_fev_siape_pensionistas_cadastro.csv carregado para a tabela 2024_fev_siape_pensionista_cadastro no BigQuery.
Arquivo 2024_mar_siape_pensionistas_cadastro.csv carregado para a tabela 2024_mar_siape_pensionista_cadastro no BigQuery.


In [123]:
# SIAPE PENSIONISTAS REMUNERAÇÃO<< 
for ano in anos:
    for mes in meses:
        # Nome do arquivo 
        nome_arquivo = f"{ano}_{mes}_siape_pensionistas_remuneracao.csv"
        caminho_arquivo = os.path.join(diretorio_remuneracao, nome_arquivo)

        # Verificar se o arquivo existe
        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            df = df.drop(df.index[-1])
            df['ANO'] = df['ANO'].astype(int)
            df = df.rename(columns={
                                    'REMUNERAÇÃO BÁSICA BRUTA (R$)': 'REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                                    'REMUNERAÇÃO BÁSICA BRUTA (U$)': 'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                                    'ABATE-TETO (R$)': 'ABATE_TETO_EM_REAIS',
                                    'ABATE-TETO (U$)': 'ABATE_TETO_EM_DOLARES',
                                    'GRATIFICAÇÃO NATALINA (R$)': 'GRATIFICACAO_NATALINA_EM_REAIS',
                                    'GRATIFICAÇÃO NATALINA (U$)': 'GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'FÉRIAS (R$)': 'FERIAS_EM_REAIS',
                                    'FÉRIAS (U$)': 'FERIAS_EM_DOLARES',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                                    'IRRF (R$)': 'IRRF_EM_REAIS',
                                    'IRRF (U$)': 'IRRF_EM_DOLARES',
                                    'PSS/RPGS (R$)': 'PSS_RPGS_EM_REAIS',
                                    'PSS/RPGS (U$)': 'PSS_RPGS_EM_DOLARES',
                                    'DEMAIS DEDUÇÕES (R$)': 'DEMAIS_DEDUCOES_EM_REAIS',
                                    'DEMAIS DEDUÇÕES (U$)': 'DEMAIS_DEDUCOES_EM_DOLARES',
                                    'PENSÃO MILITAR (R$)': 'PENSAO_MILITAR_EM_REAIS',
                                    'PENSÃO MILITAR (U$)': 'PENSAO_MILITAR_EM_DOLARES',
                                    'FUNDO DE SAÚDE (R$)': 'FUNDO_DE_SAUDE_EM_REAIS',
                                    'FUNDO DE SAÚDE (U$)': 'FUNDO_DE_SAUDE_EM_DOLARES',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                                    })
            
            #Mapeando as colunas que precisam ser convertidas para float
            colunas_float = ['REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                            'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                            'ABATE_TETO_EM_REAIS',
                            'ABATE_TETO_EM_DOLARES', 
                            'GRATIFICACAO_NATALINA_EM_REAIS',
                            'GRATIFICACAO_NATALINA_EM_DOLARES',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 
                            'FERIAS_EM_REAIS',
                            'FERIAS_EM_DOLARES', 
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                            'IRRF_EM_REAIS',
                            'IRRF_EM_DOLARES',
                            'PSS_RPGS_EM_REAIS',
                            'PSS_RPGS_EM_DOLARES',
                            'DEMAIS_DEDUCOES_EM_REAIS',
                            'DEMAIS_DEDUCOES_EM_DOLARES',
                            'PENSAO_MILITAR_EM_REAIS',
                            'PENSAO_MILITAR_EM_DOLARES',
                            'FUNDO_DE_SAUDE_EM_REAIS',
                            'FUNDO_DE_SAUDE_EM_DOLARES',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                            ]
            #Converter as colunas para float
            for coluna in colunas_float:
                df[coluna] = df[coluna].str.replace(',', '.').astype(float)
                        
            #Carregar o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_siape_pensionista_remuneracao"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema =  Schema= ([bigquery.SchemaField('ANO', 'INTEGER', description=''),
                                bigquery.SchemaField('MES', 'FLOAT', description=''),
                                bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'FLOAT', description=''),
                                bigquery.SchemaField('CPF', 'STRING', description=''),
                                bigquery.SchemaField('NOME', 'STRING', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES', 'FLOAT', description='')]
                                )
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_10172\3314104080.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_jan_siape_pensionistas_remuneracao.csv carregado para a tabela 2024_jan_siape_pensionista_remuneracao no BigQuery.


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_10172\3314104080.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_fev_siape_pensionistas_remuneracao.csv carregado para a tabela 2024_fev_siape_pensionista_remuneracao no BigQuery.


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_10172\3314104080.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')


Arquivo 2024_mar_siape_pensionistas_remuneracao.csv carregado para a tabela 2024_mar_siape_pensionista_remuneracao no BigQuery.


### BACEN

In [163]:
# BACEN PENSIONISTAS CADASTRO
for ano in anos:
    for mes_num, mes in enumerate(meses, start=1):
        nome_arquivo = f"{ano}_{mes}_bacen_pensionistas_cadastro.csv"
        caminho_arquivo = os.path.join(diretorio_cadastro, nome_arquivo)

        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            
            # Criar colunas de ano e mês de acordo com o nome do arquivo
            df['ANO'] = ano
            df['MES'] = mes_num

            # Carregando o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_bacen_pensionistas_cadastro"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema = ([
                            bigquery.SchemaField('ANO', 'INTEGER', description=''),
                            bigquery.SchemaField('MES', 'INTEGER', description=''),
                            bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'INTEGER', description=''),
                            bigquery.SchemaField('NOME', 'STRING', description=''),
                            bigquery.SchemaField('CPF', 'STRING', description=''),
                            bigquery.SchemaField('MATRICULA', 'STRING', description=''),
                            bigquery.SchemaField('CPF_REPRESENTANTE_LEGAL', 'STRING', description=''),
                            bigquery.SchemaField('NOME_REPRESENTANTE_LEGAL', 'STRING', description=''),
                            bigquery.SchemaField('CPF_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('NOME_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_TIPO_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('TIPO_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_INICIO_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DESCRICAO_CARGO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_UORG_LOTACAO_INSTITUIDOR_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('UORG_LOTACAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_ORG_LOTACAO_INSTITUIDOR_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('ORG_LOTACAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_ORGSUP_LOTACAO_INSTITUIDOR_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('ORGSUP_LOTACAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_TIPO_VINCULO', 'INTEGER', description=''),
                            bigquery.SchemaField('TIPO_VINCULO', 'STRING', description=''),
                            bigquery.SchemaField('SITUACAO_VINCULO', 'STRING', description=''),
                            bigquery.SchemaField('REGIME_JURIDICO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('JORNADA_DE_TRABALHO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_INGRESSO_CARGOFUNCAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_NOMEACAO_CARGOFUNCAO_INSTITUIDOR_PENSAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DATA_INGRESSO_ORGAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DOCUMENTO_INGRESSO_SERVICOPUBLICO_INSTITUIDOR_PENSAO', 'INTEGER', description=''),
                            bigquery.SchemaField('DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_CARGOFUNCAO_INSTITUIDOR_PENSAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_ORGAO_INSTITUIDOR_PENSAO', 'STRING', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_SERVICOPUBLICO_INSTITUIDOR_PENSAO', 'STRING', description='')
                            ])
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

Arquivo 2024_jan_bacen_pensionistas_cadastro.csv carregado para a tabela 2024_jan_bacen_pensionistas_cadastro no BigQuery.
Arquivo 2024_fev_bacen_pensionistas_cadastro.csv carregado para a tabela 2024_fev_bacen_pensionistas_cadastro no BigQuery.
Arquivo 2024_mar_bacen_pensionistas_cadastro.csv não encontrado.


In [171]:
# BACEN PENSIONISTAS REMUNERAÇÃO
for ano in anos:
    for mes in meses:
        # Nome do arquivo 
        nome_arquivo = f"{ano}_{mes}_bacen_pensionistas_remuneracao.csv"
        caminho_arquivo = os.path.join(diretorio_remuneracao, nome_arquivo)

        # Verificar se o arquivo existe
        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            df = df.drop(df.index[-1])
            df['ANO'] = df['ANO'].astype(int)
            df = df.rename(columns={
                                    'REMUNERAÇÃO BÁSICA BRUTA (R$)': 'REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                                    'REMUNERAÇÃO BÁSICA BRUTA (U$)': 'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                                    'ABATE-TETO (R$)': 'ABATE_TETO_EM_REAIS',
                                    'ABATE-TETO (U$)': 'ABATE_TETO_EM_DOLARES',
                                    'GRATIFICAÇÃO NATALINA (R$)': 'GRATIFICACAO_NATALINA_EM_REAIS',
                                    'GRATIFICAÇÃO NATALINA (U$)': 'GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'FÉRIAS (R$)': 'FERIAS_EM_REAIS',
                                    'FÉRIAS (U$)': 'FERIAS_EM_DOLARES',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                                    'IRRF (R$)': 'IRRF_EM_REAIS',
                                    'IRRF (U$)': 'IRRF_EM_DOLARES',
                                    'PSS/RPGS (R$)': 'PSS_RPGS_EM_REAIS',
                                    'PSS/RPGS (U$)': 'PSS_RPGS_EM_DOLARES',
                                    'DEMAIS DEDUÇÕES (R$)': 'DEMAIS_DEDUCOES_EM_REAIS',
                                    'DEMAIS DEDUÇÕES (U$)': 'DEMAIS_DEDUCOES_EM_DOLARES',
                                    'PENSÃO MILITAR (R$)': 'PENSAO_MILITAR_EM_REAIS',
                                    'PENSÃO MILITAR (U$)': 'PENSAO_MILITAR_EM_DOLARES',
                                    'FUNDO DE SAÚDE (R$)': 'FUNDO_DE_SAUDE_EM_REAIS',
                                    'FUNDO DE SAÚDE (U$)': 'FUNDO_DE_SAUDE_EM_DOLARES',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                                    })
            
            #Mapeando as colunas que precisam ser convertidas para float
            colunas_float = ['REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                            'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                            'ABATE_TETO_EM_REAIS',
                            'ABATE_TETO_EM_DOLARES', 
                            'GRATIFICACAO_NATALINA_EM_REAIS',
                            'GRATIFICACAO_NATALINA_EM_DOLARES',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 
                            'FERIAS_EM_REAIS',
                            'FERIAS_EM_DOLARES', 
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                            'IRRF_EM_REAIS',
                            'IRRF_EM_DOLARES',
                            'PSS_RPGS_EM_REAIS',
                            'PSS_RPGS_EM_DOLARES',
                            'DEMAIS_DEDUCOES_EM_REAIS',
                            'DEMAIS_DEDUCOES_EM_DOLARES',
                            'PENSAO_MILITAR_EM_REAIS',
                            'PENSAO_MILITAR_EM_DOLARES',
                            'FUNDO_DE_SAUDE_EM_REAIS',
                            'FUNDO_DE_SAUDE_EM_DOLARES',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                            ]
            #Converter as colunas para float
            for coluna in colunas_float:
                df[coluna] = df[coluna].str.replace(',', '.').astype(float)
                        
            #Carregar o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_bacen_pensionistas_remuneracao"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema =  Schema= ([bigquery.SchemaField('ANO', 'INTEGER', description=''),
                                bigquery.SchemaField('MES', 'FLOAT', description=''),
                                bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'FLOAT', description=''),
                                bigquery.SchemaField('CPF', 'STRING', description=''),
                                bigquery.SchemaField('NOME', 'STRING', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES', 'FLOAT', description='')]
                                )
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

Arquivo 2024_jan_bacen_pensionistas_remuneracao.csv carregado para a tabela 2024_jan_bacen_pensionistas_remuneracao no BigQuery.
Arquivo 2024_fev_bacen_pensionistas_remuneracao.csv carregado para a tabela 2024_fev_bacen_pensionistas_remuneracao no BigQuery.
Arquivo 2024_mar_bacen_pensionistas_remuneracao.csv não encontrado.


In [181]:
# BACEN ATIVOS CADASTRO
for ano in anos:
    for mes_num, mes in enumerate(meses, start=1):
        nome_arquivo = f"{ano}_{mes}_bacen_servidores_cadastro.csv"
        caminho_arquivo = os.path.join(diretorio_cadastro, nome_arquivo)

        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            df['DOCUMENTO_INGRESSO_SERVICOPUBLICO'] = df['DOCUMENTO_INGRESSO_SERVICOPUBLICO'].str.replace('00DECRETO','000000000')
            df['DOCUMENTO_INGRESSO_SERVICOPUBLICO'] = df['DOCUMENTO_INGRESSO_SERVICOPUBLICO'].astype(int)

            # Criar colunas de ano e mês de acordo com o nome do arquivo
            df['ANO'] = ano
            df['MES'] = mes_num

            # Carregando o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_bacen_servidores_cadastro"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema = [
                    bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'INTEGER', description=''),
                        bigquery.SchemaField('NOME', 'STRING', description=''),
                        bigquery.SchemaField('CPF', 'STRING', description=''),
                        bigquery.SchemaField('MATRICULA', 'STRING', description=''),
                        bigquery.SchemaField('DESCRICAO_CARGO', 'STRING', description=''),
                        bigquery.SchemaField('CLASSE_CARGO', 'STRING', description=''),
                        bigquery.SchemaField('REFERENCIA_CARGO', 'FLOAT', description=''),
                        bigquery.SchemaField('PADRAO_CARGO', 'STRING', description=''),
                        bigquery.SchemaField('NIVEL_CARGO', 'FLOAT', description=''),
                        bigquery.SchemaField('SIGLA_FUNCAO', 'STRING', description=''),
                        bigquery.SchemaField('NIVEL_FUNCAO', 'STRING', description=''),
                        bigquery.SchemaField('FUNCAO', 'STRING', description=''),
                        bigquery.SchemaField('CODIGO_ATIVIDADE', 'STRING', description=''),
                        bigquery.SchemaField('ATIVIDADE', 'STRING', description=''),
                        bigquery.SchemaField('OPCAO_PARCIAL', 'FLOAT', description=''),
                        bigquery.SchemaField('COD_UORG_LOTACAO', 'INTEGER', description=''),
                        bigquery.SchemaField('UORG_LOTACAO', 'STRING', description=''),
                        bigquery.SchemaField('COD_ORG_LOTACAO', 'INTEGER', description=''),
                        bigquery.SchemaField('ORG_LOTACAO', 'STRING', description=''),
                        bigquery.SchemaField('COD_ORGSUP_LOTACAO', 'INTEGER', description=''),
                        bigquery.SchemaField('ORGSUP_LOTACAO', 'STRING', description=''),
                        bigquery.SchemaField('COD_UORG_EXERCICIO', 'INTEGER', description=''),
                        bigquery.SchemaField('UORG_EXERCICIO', 'STRING', description=''),
                        bigquery.SchemaField('COD_ORG_EXERCICIO', 'STRING', description=''),
                        bigquery.SchemaField('ORG_EXERCICIO', 'STRING', description=''),
                        bigquery.SchemaField('COD_ORGSUP_EXERCICIO', 'INTEGER', description=''),
                        bigquery.SchemaField('ORGSUP_EXERCICIO', 'STRING', description=''),
                        bigquery.SchemaField('COD_TIPO_VINCULO', 'INTEGER', description=''),
                        bigquery.SchemaField('TIPO_VINCULO', 'STRING', description=''),
                        bigquery.SchemaField('SITUACAO_VINCULO', 'STRING', description=''),
                        bigquery.SchemaField('DATA_INICIO_AFASTAMENTO', 'STRING', description=''),
                        bigquery.SchemaField('DATA_TERMINO_AFASTAMENTO', 'STRING', description=''),
                        bigquery.SchemaField('REGIME_JURIDICO', 'STRING', description=''),
                        bigquery.SchemaField('JORNADA_DE_TRABALHO', 'STRING', description=''),
                        bigquery.SchemaField('DATA_INGRESSO_CARGOFUNCAO', 'STRING', description=''),
                        bigquery.SchemaField('DATA_NOMEACAO_CARGOFUNCAO', 'FLOAT', description=''),
                        bigquery.SchemaField('DATA_INGRESSO_ORGAO', 'STRING', description=''),
                        bigquery.SchemaField('DOCUMENTO_INGRESSO_SERVICOPUBLICO', 'INTEGER', description=''),
                        bigquery.SchemaField('DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO', 'STRING', description=''),
                        bigquery.SchemaField('DIPLOMA_INGRESSO_CARGOFUNCAO', 'FLOAT', description=''),
                        bigquery.SchemaField('DIPLOMA_INGRESSO_ORGAO', 'STRING', description=''),
                        bigquery.SchemaField('DIPLOMA_INGRESSO_SERVICOPUBLICO', 'STRING', description=''),
                        bigquery.SchemaField('UF_EXERCICIO', 'INTEGER', description=''),
                        bigquery.SchemaField('ANO', 'INTEGER', description=''),
                        bigquery.SchemaField('MES', 'INTEGER', description='')
                ]
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")


Arquivo 2024_jan_bacen_servidores_cadastro.csv carregado para a tabela 2024_jan_bacen_servidores_cadastro no BigQuery.
Arquivo 2024_fev_bacen_servidores_cadastro.csv carregado para a tabela 2024_fev_bacen_servidores_cadastro no BigQuery.
Arquivo 2024_mar_bacen_servidores_cadastro.csv não encontrado.


In [185]:
# BACEN SERVIDORES ATIVOS REMUNERAÇÃO
for mes in meses:
        # Nome do arquivo 
        nome_arquivo = f"{ano}_{mes}_bacen_servidores_remuneracao.csv"
        caminho_arquivo = os.path.join(diretorio_remuneracao, nome_arquivo)

        # Verificar se o arquivo existe
        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            df = df.drop(df.index[-1])
            df['ANO'] = df['ANO'].astype(int)
            df = df.rename(columns={
                                    'REMUNERAÇÃO BÁSICA BRUTA (R$)': 'REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                                    'REMUNERAÇÃO BÁSICA BRUTA (U$)': 'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                                    'ABATE-TETO (R$)': 'ABATE_TETO_EM_REAIS',
                                    'ABATE-TETO (U$)': 'ABATE_TETO_EM_DOLARES',
                                    'GRATIFICAÇÃO NATALINA (R$)': 'GRATIFICACAO_NATALINA_EM_REAIS',
                                    'GRATIFICAÇÃO NATALINA (U$)': 'GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'FÉRIAS (R$)': 'FERIAS_EM_REAIS',
                                    'FÉRIAS (U$)': 'FERIAS_EM_DOLARES',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                                    'IRRF (R$)': 'IRRF_EM_REAIS',
                                    'IRRF (U$)': 'IRRF_EM_DOLARES',
                                    'PSS/RPGS (R$)': 'PSS_RPGS_EM_REAIS',
                                    'PSS/RPGS (U$)': 'PSS_RPGS_EM_DOLARES',
                                    'DEMAIS DEDUÇÕES (R$)': 'DEMAIS_DEDUCOES_EM_REAIS',
                                    'DEMAIS DEDUÇÕES (U$)': 'DEMAIS_DEDUCOES_EM_DOLARES',
                                    'PENSÃO MILITAR (R$)': 'PENSAO_MILITAR_EM_REAIS',
                                    'PENSÃO MILITAR (U$)': 'PENSAO_MILITAR_EM_DOLARES',
                                    'FUNDO DE SAÚDE (R$)': 'FUNDO_DE_SAUDE_EM_REAIS',
                                    'FUNDO DE SAÚDE (U$)': 'FUNDO_DE_SAUDE_EM_DOLARES',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'

                                    })
            #Mapeando as colunas que precisam ser convertidas para float
            colunas_float = ['REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                            'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                            'ABATE_TETO_EM_REAIS',
                            'ABATE_TETO_EM_DOLARES', 
                            'GRATIFICACAO_NATALINA_EM_REAIS',
                            'GRATIFICACAO_NATALINA_EM_DOLARES',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 
                            'FERIAS_EM_REAIS',
                            'FERIAS_EM_DOLARES', 
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                            'IRRF_EM_REAIS',
                            'IRRF_EM_DOLARES',
                            'PSS_RPGS_EM_REAIS',
                            'PSS_RPGS_EM_DOLARES',
                            'DEMAIS_DEDUCOES_EM_REAIS',
                            'DEMAIS_DEDUCOES_EM_DOLARES',
                            'PENSAO_MILITAR_EM_REAIS',
                            'PENSAO_MILITAR_EM_DOLARES',
                            'FUNDO_DE_SAUDE_EM_REAIS',
                            'FUNDO_DE_SAUDE_EM_DOLARES',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                            ]
            #Converter as colunas para float
            for coluna in colunas_float:
                df[coluna] = df[coluna].str.replace(',', '.').astype(float)
                        
            #Carregar o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_bacen_servidores_remuneracao"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema =  Schema= ([bigquery.SchemaField('ANO', 'INTEGER', description=''),
                                bigquery.SchemaField('MES', 'FLOAT', description=''),
                                bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'FLOAT', description=''),
                                bigquery.SchemaField('CPF', 'STRING', description=''),
                                bigquery.SchemaField('NOME', 'STRING', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES', 'FLOAT', description='')]
                                )
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

Arquivo 2024_jan_bacen_servidores_remuneracao.csv carregado para a tabela 2024_jan_bacen_servidores_remuneracao no BigQuery.
Arquivo 2024_fev_bacen_servidores_remuneracao.csv carregado para a tabela 2024_fev_bacen_servidores_remuneracao no BigQuery.
Arquivo 2024_mar_bacen_servidores_remuneracao.csv não encontrado.


In [154]:
# BACEN APOSENTADOS CADASTRO
for ano in anos:
    for mes_num, mes in enumerate(meses, start=1):
        nome_arquivo = f"{ano}_{mes}_bacen_aposentados_cadastro.csv"
        caminho_arquivo = os.path.join(diretorio_cadastro, nome_arquivo)

        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            
            # Criar colunas de ano e mês de acordo com o nome do arquivo
            df['ANO'] = ano
            df['MES'] = mes_num

            # Carregando o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_bacen_aposentados_cadastro"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema = ([
                            bigquery.SchemaField('ANO', 'INTEGER', description=''),
                            bigquery.SchemaField('MES', 'INTEGER', description=''),
                            bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'INTEGER', description=''),
                            bigquery.SchemaField('NOME', 'STRING', description=''),
                            bigquery.SchemaField('CPF', 'STRING', description=''),
                            bigquery.SchemaField('MATRICULA', 'STRING', description=''),
                            bigquery.SchemaField('COD_TIPO_APOSENTADORIA', 'INTEGER', description=''),
                            bigquery.SchemaField('TIPO_APOSENTADORIA', 'STRING', description=''),
                            bigquery.SchemaField('DATA_APOSENTADORIA', 'STRING', description=''),
                            bigquery.SchemaField('DESCRICAO_CARGO', 'STRING', description=''),
                            bigquery.SchemaField('COD_UORG_LOTACAO', 'INTEGER', description=''),
                            bigquery.SchemaField('UORG_LOTACAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_ORG_LOTACAO', 'INTEGER', description=''),
                            bigquery.SchemaField('ORG_LOTACAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_ORGSUP_LOTACAO', 'INTEGER', description=''),
                            bigquery.SchemaField('ORGSUP_LOTACAO', 'STRING', description=''),
                            bigquery.SchemaField('COD_TIPO_VINCULO', 'INTEGER', description=''),
                            bigquery.SchemaField('TIPO_VINCULO', 'STRING', description=''),
                            bigquery.SchemaField('SITUACAO_VINCULO', 'STRING', description=''),
                            bigquery.SchemaField('REGIME_JURIDICO', 'STRING', description=''),
                            bigquery.SchemaField('JORNADA_DE_TRABALHO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_INGRESSO_CARGOFUNCAO', 'STRING', description=''),
                            bigquery.SchemaField('DATA_NOMEACAO_CARGOFUNCAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DATA_INGRESSO_ORGAO', 'STRING', description=''),
                            bigquery.SchemaField('DOCUMENTO_INGRESSO_SERVICOPUBLICO', 'INTEGER', description=''),
                            bigquery.SchemaField('DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO', 'STRING', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_CARGOFUNCAO', 'FLOAT', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_ORGAO', 'STRING', description=''),
                            bigquery.SchemaField('DIPLOMA_INGRESSO_SERVICOPUBLICO', 'STRING', description='')
                            ])
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

Arquivo 2024_jan_bacen_aposentados_cadastro.csv carregado para a tabela 2024_jan_bacen_aposentados_cadastro no BigQuery.
Arquivo 2024_fev_bacen_aposentados_cadastro.csv carregado para a tabela 2024_fev_bacen_aposentados_cadastro no BigQuery.
Arquivo 2024_mar_bacen_aposentados_cadastro.csv não encontrado.


In [158]:
# BACEN APOSENTADOS REMUNERAÇÃO<< 
for ano in anos:
    for mes in meses:
        # Nome do arquivo 
        nome_arquivo = f"{ano}_{mes}_bacen_aposentados_remuneracao.csv"
        caminho_arquivo = os.path.join(diretorio_remuneracao, nome_arquivo)

        # Verificar se o arquivo existe
        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            df = df.drop(df.index[-1])
            df['ANO'] = df['ANO'].astype(int)
            df = df.rename(columns={
                                    'REMUNERAÇÃO BÁSICA BRUTA (R$)': 'REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                                    'REMUNERAÇÃO BÁSICA BRUTA (U$)': 'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                                    'ABATE-TETO (R$)': 'ABATE_TETO_EM_REAIS',
                                    'ABATE-TETO (U$)': 'ABATE_TETO_EM_DOLARES',
                                    'GRATIFICAÇÃO NATALINA (R$)': 'GRATIFICACAO_NATALINA_EM_REAIS',
                                    'GRATIFICAÇÃO NATALINA (U$)': 'GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                                    'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)': 'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES',
                                    'FÉRIAS (R$)': 'FERIAS_EM_REAIS',
                                    'FÉRIAS (U$)': 'FERIAS_EM_DOLARES',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                                    'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)': 'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                                    'IRRF (R$)': 'IRRF_EM_REAIS',
                                    'IRRF (U$)': 'IRRF_EM_DOLARES',
                                    'PSS/RPGS (R$)': 'PSS_RPGS_EM_REAIS',
                                    'PSS/RPGS (U$)': 'PSS_RPGS_EM_DOLARES',
                                    'DEMAIS DEDUÇÕES (R$)': 'DEMAIS_DEDUCOES_EM_REAIS',
                                    'DEMAIS DEDUÇÕES (U$)': 'DEMAIS_DEDUCOES_EM_DOLARES',
                                    'PENSÃO MILITAR (R$)': 'PENSAO_MILITAR_EM_REAIS',
                                    'PENSÃO MILITAR (U$)': 'PENSAO_MILITAR_EM_DOLARES',
                                    'FUNDO DE SAÚDE (R$)': 'FUNDO_DE_SAUDE_EM_REAIS',
                                    'FUNDO DE SAÚDE (U$)': 'FUNDO_DE_SAUDE_EM_DOLARES',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                                    'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)': 'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                                    'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)': 'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*)': 'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                                    'VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$)': 'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                                    'TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)': 'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                                    })
            
            #Mapeando as colunas que precisam ser convertidas para float
            colunas_float = ['REMUNERACAO_BASICA_BRUTA_EM_REAIS',
                            'REMUNERACAO_BASICA_BRUTA_EM_DOLARES',
                            'ABATE_TETO_EM_REAIS',
                            'ABATE_TETO_EM_DOLARES', 
                            'GRATIFICACAO_NATALINA_EM_REAIS',
                            'GRATIFICACAO_NATALINA_EM_DOLARES',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS',
                            'ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 
                            'FERIAS_EM_REAIS',
                            'FERIAS_EM_DOLARES', 
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS',
                            'OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES',
                            'IRRF_EM_REAIS',
                            'IRRF_EM_DOLARES',
                            'PSS_RPGS_EM_REAIS',
                            'PSS_RPGS_EM_DOLARES',
                            'DEMAIS_DEDUCOES_EM_REAIS',
                            'DEMAIS_DEDUCOES_EM_DOLARES',
                            'PENSAO_MILITAR_EM_REAIS',
                            'PENSAO_MILITAR_EM_DOLARES',
                            'FUNDO_DE_SAUDE_EM_REAIS',
                            'FUNDO_DE_SAUDE_EM_DOLARES',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS',
                            'TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS',
                            'REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS',
                            'VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS',
                            'TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES'
                            ]
            #Converter as colunas para float
            for coluna in colunas_float:
                df[coluna] = df[coluna].str.replace(',', '.').astype(float)
                        
            #Carregar o DataFrame para o BigQuery
            nome_tabela = f"{ano}_{mes}_bacen_aposentados_remuneracao"
            dataset_ref = client.dataset('portal_transparencia_cgu')
            table_ref = dataset_ref.table(nome_tabela)
            schema =  Schema= ([bigquery.SchemaField('ANO', 'INTEGER', description=''),
                                bigquery.SchemaField('MES', 'FLOAT', description=''),
                                bigquery.SchemaField('Id_SERVIDOR_PORTAL', 'FLOAT', description=''),
                                bigquery.SchemaField('CPF', 'STRING', description=''),
                                bigquery.SchemaField('NOME', 'STRING', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_BASICA_BRUTA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('ABATE_TETO_DA_GRATIFICACAO_NATALINA_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FERIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('OUTRAS_REMUNERACOES_EVENTUAIS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('IRRF_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PSS_RPGS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('DEMAIS_DEDUCOES_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('PENSAO_MILITAR_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('FUNDO_DE_SAUDE_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TAXA_DE_OCUPACAO_IMOVEL_FUNCIONAL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('REMUNERACAO_APOS_DEDUCOES_OBRIGATORIAS_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_CIVIL_EM_DOLARES', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_REGISTRADAS_EM_SISTEMAS_DE_PESSOAL_MILITAR_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('VERBAS_INDENIZATORIAS_PROGRAMA_DESLIGAMENTO_VOLUNTARIO_MP_792_2017_EM_DOLARES','FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_REAIS', 'FLOAT', description=''),
                                bigquery.SchemaField('TOTAL_DE_VERBAS_INDENIZATORIAS_EM_DOLARES', 'FLOAT', description='')]
                                )
        
            job_config = bigquery.LoadJobConfig(schema=schema)
            job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
            job.result()  

            print(f'Arquivo {nome_arquivo} carregado para a tabela {nome_tabela} no BigQuery.')
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

Arquivo 2024_jan_bacen_aposentados_remuneracao.csv carregado para a tabela 2024_jan_bacen_aposentados_remuneracao no BigQuery.
Arquivo 2024_fev_bacen_aposentados_remuneracao.csv carregado para a tabela 2024_fev_bacen_aposentados_remuneracao no BigQuery.
Arquivo 2024_mar_bacen_aposentados_remuneracao.csv não encontrado.


# Verificando duplicatas

In [4]:
diretorio_cadastro = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados novos\\CGU\\BACEN\\SERVIDORES_ATIVOS_CADASTRO'

In [3]:
diretorio_cadastro = 'G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Dados novos\\\\CGU\\\\MILITARES\\\\Militares_ativos\\Cadastro'


In [ ]:
os.listdir(diretorio_cadastro)

In [4]:
os.listdir(diretorio_cadastro)

['2023_mai_militares_ativos_cadastro.csv',
 '2023_jun_militares_ativos_cadastro.csv',
 '2023_jul_militares_ativos_cadastro.csv',
 '2023_ago_militares_ativos_cadastro.csv',
 '2023_set_militares_ativos_cadastro.csv',
 '2023_out_militares_ativos_cadastro.csv',
 '2023_nov_militares_ativos_cadastro.csv',
 '2023_dez_militares_ativos_cadastro.csv',
 '2013_jan_militares_ativos_cadastro.csv',
 '2013_fev_militares_ativos_cadastro.csv',
 '2013_mar_militares_ativos_cadastro.csv',
 '2013_abr_militares_ativos_cadastro.csv',
 '2013_mai_militares_ativos_cadastro.csv',
 '2013_jun_militares_ativos_cadastro.csv',
 '2013_jul_militares_ativos_cadastro.csv',
 '2013_ago_militares_ativos_cadastro.csv',
 '2013_set_militares_ativos_cadastro.csv',
 '2013_out_militares_ativos_cadastro.csv',
 '2013_nov_militares_ativos_cadastro.csv',
 '2013_dez_militares_ativos_cadastro.csv',
 '2014_jan_militares_ativos_cadastro.csv',
 '2014_fev_militares_ativos_cadastro.csv',
 '2014_mar_militares_ativos_cadastro.csv',
 '2014_abr_

### Loop para verificar duplicada em cadastro no ID e NOME + CPF

In [6]:
anos = range(2013, 2024)
meses = ['jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez']

In [ ]:
for ano in anos:
    for mes in meses:
        # Nome do arquivo 
        nome_arquivo = f"{ano}_{mes}_militares_ativos_cadastro.csv"
        caminho_arquivo = os.path.join(diretorio_cadastro, nome_arquivo)

        # Verificar se o arquivo existe
        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            duplicatas_id = list(df.duplicated(subset=['Id_SERVIDOR_PORTAL'])) 
        
            df['duplicado_id']=duplicatas_id
            print(f"No arquivo {nome_arquivo} consta: {df['duplicado_id'].value_counts()}")


In [ ]:
for ano in anos:
    for mes in meses:
        # Nome do arquivo 
        nome_arquivo = f"{ano}_{mes}_militares_ativos_cadastro.csv"
        caminho_arquivo = os.path.join(diretorio_cadastro, nome_arquivo)

        # Verificar se o arquivo existe
        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            duplicatas_nomecpf = list(df.duplicated(subset=['NOME', 'CPF']))
            duplicatas_nomecpf
        
            df['duplicado_id']=duplicatas_nomecpf
            print(f"No arquivo {nome_arquivo} consta: {df['duplicado_id'].value_counts()}")


### Loop para verificar duplicada em remuneração no ID e NOME + CPF

In [25]:
diretorio_remuneracao = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados novos\\CGU\\BACEN\\SERVIDORES_ATIVOS_REMUNERACAO'

In [26]:
os.chdir(diretorio_remuneracao)

In [27]:
os.listdir(diretorio_remuneracao)

['2013_jan_bacen_servidores_remuneracao.csv',
 '2013_fev_bacen_servidores_remuneracao.csv',
 '2013_mar_bacen_servidores_remuneracao.csv',
 '2013_abr_bacen_servidores_remuneracao.csv',
 '2013_mai_bacen_servidores_remuneracao.csv',
 '2013_jun_bacen_servidores_remuneracao.csv',
 '2013_jul_bacen_servidores_remuneracao.csv',
 '2013_ago_bacen_servidores_remuneracao.csv',
 '2013_set_bacen_servidores_remuneracao.csv',
 '2013_out_bacen_servidores_remuneracao.csv',
 '2013_nov_bacen_servidores_remuneracao.csv',
 '2013_dez_bacen_servidores_remuneracao.csv',
 '2014_jan_bacen_servidores_remuneracao.csv',
 '2014_fev_bacen_servidores_remuneracao.csv',
 '2014_mar_bacen_servidores_remuneracao.csv',
 '2014_abr_bacen_servidores_remuneracao.csv',
 '2014_mai_bacen_servidores_remuneracao.csv',
 '2014_jun_bacen_servidores_remuneracao.csv',
 '2014_jul_bacen_servidores_remuneracao.csv',
 '2014_ago_bacen_servidores_remuneracao.csv',
 '2014_set_bacen_servidores_remuneracao.csv',
 '2014_out_bacen_servidores_remune

In [28]:
for ano in anos:
    for mes in meses:
        # Nome do arquivo 
        nome_arquivo = f"{ano}_{mes}_bacen_servidores_remuneracao.csv"
        caminho_arquivo = os.path.join(diretorio_remuneracao, nome_arquivo)

        # Verificar se o arquivo existe
        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            duplicatas_nomecpf = list(df.duplicated(subset=['NOME', 'CPF']))
            duplicatas_nomecpf
        
            df['duplicado_id']=duplicatas_nomecpf
            print(f"No arquivo {nome_arquivo} consta: {df['duplicado_id'].value_counts()}")


No arquivo 2013_jan_bacen_servidores_remuneracao.csv consta: False    4339
Name: duplicado_id, dtype: int64
No arquivo 2013_fev_bacen_servidores_remuneracao.csv consta: False    4308
Name: duplicado_id, dtype: int64
No arquivo 2013_mar_bacen_servidores_remuneracao.csv consta: False    4287
Name: duplicado_id, dtype: int64
No arquivo 2013_abr_bacen_servidores_remuneracao.csv consta: False    4258
Name: duplicado_id, dtype: int64
No arquivo 2013_mai_bacen_servidores_remuneracao.csv consta: False    4233
Name: duplicado_id, dtype: int64
No arquivo 2013_jun_bacen_servidores_remuneracao.csv consta: False    4205
Name: duplicado_id, dtype: int64
No arquivo 2013_jul_bacen_servidores_remuneracao.csv consta: False    4180
Name: duplicado_id, dtype: int64
No arquivo 2013_ago_bacen_servidores_remuneracao.csv consta: False    4131
Name: duplicado_id, dtype: int64
No arquivo 2013_set_bacen_servidores_remuneracao.csv consta: False    4098
Name: duplicado_id, dtype: int64
No arquivo 2013_out_bacen_se

In [29]:
for ano in anos:
    for mes in meses:
        # Nome do arquivo 
        nome_arquivo = f"{ano}_{mes}_bacen_servidores_remuneracao.csv"
        caminho_arquivo = os.path.join(diretorio_remuneracao, nome_arquivo)

        # Verificar se o arquivo existe
        if os.path.exists(caminho_arquivo):
            # Carregar o arquivo CSV para um DataFrame
            df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1')
            duplicatas_id = list(df.duplicated(subset=['Id_SERVIDOR_PORTAL'])) 
        
            df['duplicado_id']=duplicatas_id
            print(f"No arquivo {nome_arquivo} consta: {df['duplicado_id'].value_counts()}")


No arquivo 2013_jan_bacen_servidores_remuneracao.csv consta: False    4339
Name: duplicado_id, dtype: int64
No arquivo 2013_fev_bacen_servidores_remuneracao.csv consta: False    4308
Name: duplicado_id, dtype: int64
No arquivo 2013_mar_bacen_servidores_remuneracao.csv consta: False    4287
Name: duplicado_id, dtype: int64
No arquivo 2013_abr_bacen_servidores_remuneracao.csv consta: False    4258
Name: duplicado_id, dtype: int64
No arquivo 2013_mai_bacen_servidores_remuneracao.csv consta: False    4233
Name: duplicado_id, dtype: int64
No arquivo 2013_jun_bacen_servidores_remuneracao.csv consta: False    4205
Name: duplicado_id, dtype: int64
No arquivo 2013_jul_bacen_servidores_remuneracao.csv consta: False    4180
Name: duplicado_id, dtype: int64
No arquivo 2013_ago_bacen_servidores_remuneracao.csv consta: False    4131
Name: duplicado_id, dtype: int64
No arquivo 2013_set_bacen_servidores_remuneracao.csv consta: False    4098
Name: duplicado_id, dtype: int64
No arquivo 2013_out_bacen_se

In [16]:
df=pd.read_csv('2022_ago_bacen_pensionistas_cadastro.csv', sep=';', encoding = 'latin-1')
df

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,CPF_REPRESENTANTE_LEGAL,NOME_REPRESENTANTE_LEGAL,CPF_INSTITUIDOR_PENSAO,NOME_INSTITUIDOR_PENSAO,COD_TIPO_PENSAO,TIPO_PENSAO,...,REGIME_JURIDICO_INSTITUIDOR_PENSAO,JORNADA_DE_TRABALHO_INSTITUIDOR_PENSAO,DATA_INGRESSO_CARGOFUNCAO_INSTITUIDOR_PENSAO,DATA_NOMEACAO_CARGOFUNCAO_INSTITUIDOR_PENSAO,DATA_INGRESSO_ORGAO_INSTITUIDOR_PENSAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO_INSTITUIDOR_PENSAO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO_INSTITUIDOR_PENSAO,DIPLOMA_INGRESSO_CARGOFUNCAO_INSTITUIDOR_PENSAO,DIPLOMA_INGRESSO_ORGAO_INSTITUIDOR_PENSAO,DIPLOMA_INGRESSO_SERVICOPUBLICO_INSTITUIDOR_PENSAO
0,2627712,ABEL AMBROSIO DA SILVA FILHO,***.210.784-**,044****,NaN,NaN,***.829.231-**,IRLENE OLIVEIRA DE SOUZA,8,(Ex) cônjuge / (Ex) companheiro (a) / Viúvo (a),...,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,08/05/1972,NaN,NaN,0,08/05/1972,NaN,TERMO DE OPCAO,TERMO DE OPCAO
1,2714273,ADILSON MOREIRA DE CARVALHO,***.369.557-**,077****,NaN,NaN,***.505.557-**,OLENDINA ZANETTE CASTEGLIONE,10,(Ex) cônjuge / (Ex) companheiro (a) / Viúvo (a),...,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,02/06/1977,NaN,NaN,0,02/06/1977,NaN,NaN,NaN
2,2144198,ADNILSA FERREIRA PESSOA,***.618.494-**,068****,NaN,NaN,***.042.664-**,MARCOS TULIO CICERO PESSOA,38,(Ex) cônjuge / (Ex) companheiro (a) / Viúvo (a),...,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,09/01/1975,NaN,NaN,0,09/01/1975,NaN,NaN,NaN
3,3695576,ADONAY CONCEICAO DE OLIVEIRA BARCOS,***.924.868-**,080****,NaN,NaN,***.702.158-**,PAULO BARCOS,8,(Ex) cônjuge / (Ex) companheiro (a) / Viúvo (a),...,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,20/07/1977,NaN,NaN,0,20/07/1977,NaN,NaN,NaN
4,3774973,ADRIA RODRIGUES DA COSTA,***.639.602-**,025****,NaN,NaN,***.697.441-**,DOMINGOS ALVES DA COSTA,36,(Ex) cônjuge / (Ex) companheiro (a) / Viúvo (a),...,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,15/02/1977,NaN,NaN,0,15/02/1977,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,2756003,*** PENSIONISTA MENOR DE 16 ANOS ***,NaN,050****,NaN,NaN,***.429.961-**,JOAQUIM BORGES PINTO,13,Filho (a) / Menor sob guarda / Enteado (a),...,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,18/03/1980,NaN,NaN,0,18/03/1980,NaN,NaN,NaN
925,2796792,*** PENSIONISTA MENOR DE 16 ANOS ***,NaN,021****,NaN,NaN,***.240.058-**,CLOVIS VIDAL POLETO,20,Filho (a) / Menor sob guarda / Enteado (a),...,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,10/05/2004,NaN,12/04/2004,26228,10/05/2004,NaN,PORTARIA,PORTARIA
926,3306910,*** PENSIONISTA MENOR DE 16 ANOS ***,NaN,062****,NaN,NaN,***.015.751-**,LINDOMAR PEDRO CAMARGO,44,Filho (a) / Menor sob guarda / Enteado (a),...,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,07/08/1979,NaN,NaN,0,07/08/1979,NaN,NaN,NaN
927,3336113,*** PENSIONISTA MENOR DE 16 ANOS ***,NaN,020****,NaN,NaN,***.642.021-**,CLEANTO ARAUJO,13,Filho (a) / Menor sob guarda / Enteado (a),...,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,13/09/1974,NaN,NaN,0,13/09/1974,NaN,NaN,NaN


In [18]:
duplicatas = list(df.duplicated(subset=['Id_SERVIDOR_PORTAL']))
duplicatas

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [19]:
df['duplicado']=duplicatas 

In [20]:
df['duplicado'].value_counts()

False    929
Name: duplicado, dtype: int64

## Verificação de parâmetros para loop

In [40]:
diretorio_remuneracao = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados novos\\CGU\\BACEN\\SERVIDORES_ATIVOS_REMUNERACAO'

In [41]:
os.listdir(diretorio_remuneracao)

['2013_jan_bacen_servidores_remuneracao.csv',
 '2013_fev_bacen_servidores_remuneracao.csv',
 '2013_mar_bacen_servidores_remuneracao.csv',
 '2013_abr_bacen_servidores_remuneracao.csv',
 '2013_mai_bacen_servidores_remuneracao.csv',
 '2013_jun_bacen_servidores_remuneracao.csv',
 '2013_jul_bacen_servidores_remuneracao.csv',
 '2013_ago_bacen_servidores_remuneracao.csv',
 '2013_set_bacen_servidores_remuneracao.csv',
 '2013_out_bacen_servidores_remuneracao.csv',
 '2013_nov_bacen_servidores_remuneracao.csv',
 '2013_dez_bacen_servidores_remuneracao.csv',
 '2014_jan_bacen_servidores_remuneracao.csv',
 '2014_fev_bacen_servidores_remuneracao.csv',
 '2014_mar_bacen_servidores_remuneracao.csv',
 '2014_abr_bacen_servidores_remuneracao.csv',
 '2014_mai_bacen_servidores_remuneracao.csv',
 '2014_jun_bacen_servidores_remuneracao.csv',
 '2014_jul_bacen_servidores_remuneracao.csv',
 '2014_ago_bacen_servidores_remuneracao.csv',
 '2014_set_bacen_servidores_remuneracao.csv',
 '2014_out_bacen_servidores_remune

In [42]:
df = pd.read_csv('2018_mar_bacen_servidores_remuneracao.csv', sep=';', encoding='latin-1')
df

,ANO,MES,Id_SERVIDOR_PORTAL,CPF,NOME,REMUNERAÇÃO BÁSICA BRUTA (R$),REMUNERAÇÃO BÁSICA BRUTA (U$),ABATE-TETO (R$),ABATE-TETO (U$),GRATIFICAÇÃO NATALINA (R$),...,REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$),REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*),VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$),VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$),TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*),TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)
0,2018,3.0,3277279.0,***.056.281-**,AARAO DIAMANTINO OLIVEIRA,"29077,59","0,00","0,00","0,00","0,00",...,"19949,54","0,00","458,00","0,00","0,00","0,00","0,00","0,00","458,00","0,00"
1,2018,3.0,1665453.0,***.297.631-**,ABDIAS CASSIO MARROCOS GOMES,"16398,90","0,00","0,00","0,00","0,00",...,"11871,89","0,00","458,00","0,00","0,00","0,00","0,00","0,00","458,00","0,00"
2,2018,3.0,1334423.0,***.347.385-**,ABEL ALFREDO FILHO,"28577,62","0,00","0,00","0,00","0,00",...,"19534,92","0,00","458,00","0,00","0,00","0,00","0,00","0,00","458,00","0,00"
3,2018,3.0,1352756.0,***.305.318-**,ABEL CARLOS RODRIGUES,"25745,60","0,00","0,00","0,00","0,00",...,"17481,71","0,00","458,00","0,00","0,00","0,00","0,00","0,00","458,00","0,00"
4,2018,3.0,3263691.0,***.856.141-**,ABIAS PEREIRA DA SILVA,"31909,61","0,00","0,00","0,00","0,00",...,"22002,75","0,00","458,00","0,00","0,00","0,00","0,00","0,00","458,00","0,00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3830,2018,3.0,1209240.0,***.870.954-**,ZHAN HAI XING,"25745,60","0,00","0,00","0,00","0,00",...,"17585,98","0,00","458,00","0,00","0,00","0,00","0,00","0,00","458,00","0,00"
3831,2018,3.0,1232850.0,***.391.211-**,ZULEICA DE CAMPOS SFREDO,"13066,91","0,00","0,00","0,00","0,00",...,"9508,33","0,00","458,00","0,00","0,00","0,00","0,00","0,00","458,00","0,00"
3832,2018,3.0,812847.0,***.289.826-**,ZULMAR DA SILVA PEIXOTO,"25745,60","0,00","0,00","0,00","0,00",...,"17638,12","0,00","458,00","0,00","0,00","0,00","0,00","0,00","458,00","0,00"
3833,2018,3.0,1521836.0,***.113.787-**,ZULMAR MAYETA,"31909,61","0,00","0,00","0,00","0,00",...,"21950,62","0,00","458,00","0,00","0,00","0,00","0,00","0,00","458,00","0,00"


In [33]:
df.columns

Index(['ANO', 'MES', 'Id_SERVIDOR_PORTAL', 'CPF', 'NOME',
       'REMUNERAÇÃO BÁSICA BRUTA (R$)', 'REMUNERAÇÃO BÁSICA BRUTA (U$)',
       'ABATE-TETO (R$)', 'ABATE-TETO (U$)', 'GRATIFICAÇÃO NATALINA (R$)',
       'GRATIFICAÇÃO NATALINA (U$)',
       'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)',
       'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)', 'FÉRIAS (R$)',
       'FÉRIAS (U$)', 'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)',
       'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)', 'IRRF (R$)', 'IRRF (U$)',
       'PSS/RPGS (R$)', 'PSS/RPGS (U$)', 'DEMAIS DEDUÇÕES (R$)',
       'DEMAIS DEDUÇÕES (U$)', 'PENSÃO MILITAR (R$)', 'PENSÃO MILITAR (U$)',
       'FUNDO DE SAÚDE (R$)', 'FUNDO DE SAÚDE (U$)',
       'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)',
       'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)',
       'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)',
       'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)',
       'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)',
       'VERBAS INDENIZATÓ

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3835 entries, 0 to 3834
Data columns (total 39 columns):
 #   Column                                                                      Non-Null Count  Dtype  
---  ------                                                                      --------------  -----  
 0   ANO                                                                         3835 non-null   object 
 1   MES                                                                         3834 non-null   float64
 2   Id_SERVIDOR_PORTAL                                                          3834 non-null   float64
 3   CPF                                                                         3834 non-null   object 
 4   NOME                                                                        3834 non-null   object 
 5   REMUNERAÇÃO BÁSICA BRUTA (R$)                                               3834 non-null   object 
 6   REMUNERAÇÃO BÁSICA BRUTA (U$)                   

# Teste duplicatas

In [41]:
diretorio = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados novos\\CGU\\SIAPE\\SERVIDORES_ATIVOS_CADASTRO'

In [42]:
os.chdir(diretorio)

In [68]:
os.listdir(diretorio)

['2013_jan_siape_servidores_cadastro.csv',
 '2013_fev_siape_servidores_cadastro.csv',
 '2013_mar_siape_servidores_cadastro.csv',
 '2013_abr_siape_servidores_cadastro.csv',
 '2013_mai_siape_servidores_cadastro.csv',
 '2013_jun_siape_servidores_cadastro.csv',
 '2013_jul_siape_servidores_cadastro.csv',
 '2013_ago_siape_servidores_cadastro.csv',
 '2013_set_siape_servidores_cadastro.csv',
 '2013_out_siape_servidores_cadastro.csv',
 '2013_nov_siape_servidores_cadastro.csv',
 '2013_dez_siape_servidores_cadastro.csv',
 '2014_jan_siape_servidores_cadastro.csv',
 '2014_fev_siape_servidores_cadastro.csv',
 '2014_mar_siape_servidores_cadastro.csv',
 '2014_abr_siape_servidores_cadastro.csv',
 '2014_mai_siape_servidores_cadastro.csv',
 '2014_jun_siape_servidores_cadastro.csv',
 '2014_jul_siape_servidores_cadastro.csv',
 '2014_ago_siape_servidores_cadastro.csv',
 '2014_set_siape_servidores_cadastro.csv',
 '2014_out_siape_servidores_cadastro.csv',
 '2014_nov_siape_servidores_cadastro.csv',
 '2014_dez_

In [43]:
df = pd.read_csv('2023_nov_siape_servidores_cadastro.csv', sep=';', encoding='latin-1')


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_8316\635804932.py:1: DtypeWarning: Columns (2,5,7,9,34,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  base = pd.read_csv('2023_nov_siape_servidores_cadastro.csv', sep=';', encoding='latin-1')


In [9]:
df = df.drop(df.index[df['NOME']=='Sigiloso'])

In [10]:
## Aqui tem tanto função gratificadas como também os símbolos dos cargos em comissão. 
df['SIGLA_FUNCAO'].unique()

array(['-1', 'FG', 'CCX', 'CD', 'GF', 'FEX', 'FGR', 'FUC', 'CCT', 'GSE',
       'PCC', 'FAA', 'ETE', 'CGE', 'RMA', 'PFC', 'PFG', 'RMP', 'FGS',
       'GFE', 'CAS', 'RGA', 'CTE', 'CA', 'GTS', 'FTS', 'EC', 'CCD', 'FGT',
       'GNE', 'FGP', 'FGC', 'FCT', 'DAS', 'FCE', 'FGD', 'FPE', 'FCC',
       'GFC', '-11', -11], dtype=object)

In [13]:
funcoes = ['FG', 'CCX', 'CD', 'GF', 'FEX', 'FGR', 'FUC', 'CCT', 'GSE',
       'PCC', 'FAA', 'ETE', 'CGE', 'RMA', 'PFC', 'PFG', 'RMP', 'FGS',
       'GFE', 'CAS', 'RGA', 'CTE', 'CA', 'GTS', 'FTS', 'EC', 'CCD', 'FGT',
       'GNE', 'FGP', 'FGC', 'FCT', 'DAS', 'FCE', 'FGD', 'FPE', 'FCC',
       'GFC']

In [77]:
## Verifica se vínculos com cargos em comissão ou função possui a especificação da descrição do cargo. 
# O resultado é que não, portanto, provavelmente duplica-se por conta dos desses cargos. 

df[df['SIGLA_FUNCAO'].isin(funcoes)]['DESCRICAO_CARGO'].unique()
# 

array(['Sem informaç'], dtype=object)

In [10]:

duplicatas_id = list(df.duplicated(subset=['Id_SERVIDOR_PORTAL'], keep=False)) 
df['duplicado_id']=duplicatas_id


In [80]:
df

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,DESCRICAO_CARGO,CLASSE_CARGO,REFERENCIA_CARGO,PADRAO_CARGO,NIVEL_CARGO,SIGLA_FUNCAO,NIVEL_FUNCAO,FUNCAO,CODIGO_ATIVIDADE,ATIVIDADE,OPCAO_PARCIAL,COD_UORG_LOTACAO,UORG_LOTACAO,COD_ORG_LOTACAO,ORG_LOTACAO,COD_ORGSUP_LOTACAO,ORGSUP_LOTACAO,COD_UORG_EXERCICIO,UORG_EXERCICIO,COD_ORG_EXERCICIO,ORG_EXERCICIO,COD_ORGSUP_EXERCICIO,ORGSUP_EXERCICIO,COD_TIPO_VINCULO,TIPO_VINCULO,SITUACAO_VINCULO,DATA_INICIO_AFASTAMENTO,DATA_TERMINO_AFASTAMENTO,REGIME_JURIDICO,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,UF_EXERCICIO,duplicado_id
0,310606473,AARAO CARLOS LUZ MACAMBIRA,***.017.623-**,016****,BIBLIOTECARIO-DOCUMENTALISTA,E,0.0,410,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,26405000000040,DIRETORIA GERAL/CAMPUS SOBRAL,26405,Instituto Federal do Ceará,15000,Ministério da Educação,26405000000717,DIRETORIA DE ENSINO-SOB,26405,Instituto Federal do Ceará,15000,Ministério da Educação,2,Cargo,ATIVO PERMANENTE,NaN,NaN,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,11/02/2009,NaN,29/12/2008,699,11/02/2009,NaN,LEI,PORTARIA,-1,False
1,313006968,AARAO FERREIRA LIMA NETO,***.116.132-**,014****,Sem informaç,NaN,-1.0,-1,-1.0,FG,1,FUNCAO GRATIFICADA - IFES,99,DIRETOR(A),NaN,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,Universidade Federal do Pará,15000,Ministério da Educação,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,Universidade Federal do Pará,15000,Ministério da Educação,1,Função,ATIVO PERMANENTE,NaN,NaN,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,16/03/2020,NaN,24/07/2006,2475,16/08/2006,NaN,PORTARIA,PORTARIA,-1,True
2,313006968,AARAO FERREIRA LIMA NETO,***.116.132-**,014****,PROFESSOR DO MAGISTERIO SUPERIOR,7,0.0,NaN,702.0,-1,-1,Sem informação,-1,Sem informaç,NaN,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,Universidade Federal do Pará,15000,Ministério da Educação,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,Universidade Federal do Pará,15000,Ministério da Educação,2,Cargo,ATIVO PERMANENTE,NaN,NaN,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,01/03/2013,NaN,24/07/2006,2475,16/08/2006,NaN,PORTARIA,PORTARIA,-1,True
3,311039523,AARAO MEIR SERRUYA,***.693.832-**,033****,FISIOTERAPEUTA - 30H,S,0.0,NaN,201.0,-1,-1,Sem informação,-1,Sem informaç,NaN,26443035000000,COMPLEXO HOSPITALAR DA UFPA,26443,Empresa Brasileira de Serviços Hospitalares,15000,Ministério da Educação,-3,Inválido,26443,Empresa Brasileira de Serviços Hospitalares,15000,Ministério da Educação,2,Cargo,CELETISTA/EMPREGADO,NaN,NaN,CONSOLIDACAO DAS LEIS DO TRABALHO,30 HORAS SEMANAIS,07/12/2022,NaN,07/12/2022,NaN,NaN,NaN,CONTRATO,Inválido,-1,False
4,310150191,AARAO PEREIRA DE ARAUJO JUNIOR,***.031.184-**,002****,PROFESSOR ENS BASICO TECN TECNOLOGICO,D,0.0,NaN,501.0,-1,-1,Sem informação,-1,Sem informaç,NaN,26417000000013,DIR. DESENVOLVIMENTO ENSINO-JP,26417,Instituto Federal da Paraíba,15000,Ministério da Educação,26417000000240,"UNID. ACAD. I DES, ESTR.M.AMB-JP",26417,Instituto Federal da Paraíba,15000,Ministério da Educação,2,Cargo,ATIVO PERMANENTE,NaN,NaN,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,01/03/2013,NaN,29/12/2008,000000124,03/05/1993,NaN,LEI,PORTARIA,-1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711995,311249635,ZUZITA DOS SANTOS VIANA,***.041.463-**,004****,PROFESSOR DE 1 GRAU,S,0.0,III,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,30202000002000,SV APOIO ADMINISTRATIVO/CR-MA,30202,Fundação Nacional do Índio,17200,MINISTERIO DOS POVOS INDIGENAS,30202000002000,SV APOIO ADMINISTRATIVO/CR-MA,30202,Fundação Nacional do Índio,17200,MINISTERIO DOS POVOS INDIGENAS,2,Cargo,ATIVO PERMANENTE,NaN,NaN,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/07/2006,NaN,01/08/198

In [79]:
df['duplicado_id'].value_counts()

False    513649
True     198351
Name: duplicado_id, dtype: int64

In [11]:
df = df[df['duplicado_id']==True]

In [83]:
df

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,DESCRICAO_CARGO,CLASSE_CARGO,REFERENCIA_CARGO,PADRAO_CARGO,NIVEL_CARGO,SIGLA_FUNCAO,NIVEL_FUNCAO,FUNCAO,CODIGO_ATIVIDADE,ATIVIDADE,OPCAO_PARCIAL,COD_UORG_LOTACAO,UORG_LOTACAO,COD_ORG_LOTACAO,ORG_LOTACAO,COD_ORGSUP_LOTACAO,ORGSUP_LOTACAO,COD_UORG_EXERCICIO,UORG_EXERCICIO,COD_ORG_EXERCICIO,ORG_EXERCICIO,COD_ORGSUP_EXERCICIO,ORGSUP_EXERCICIO,COD_TIPO_VINCULO,TIPO_VINCULO,SITUACAO_VINCULO,DATA_INICIO_AFASTAMENTO,DATA_TERMINO_AFASTAMENTO,REGIME_JURIDICO,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,UF_EXERCICIO,duplicado_id
1,313006968,AARAO FERREIRA LIMA NETO,***.116.132-**,014****,Sem informaç,NaN,-1.0,-1,-1.0,FG,1,FUNCAO GRATIFICADA - IFES,99,DIRETOR(A),NaN,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,Universidade Federal do Pará,15000,Ministério da Educação,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,Universidade Federal do Pará,15000,Ministério da Educação,1,Função,ATIVO PERMANENTE,NaN,NaN,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,16/03/2020,NaN,24/07/2006,2475,16/08/2006,NaN,PORTARIA,PORTARIA,-1,True
2,313006968,AARAO FERREIRA LIMA NETO,***.116.132-**,014****,PROFESSOR DO MAGISTERIO SUPERIOR,7,0.0,NaN,702.0,-1,-1,Sem informação,-1,Sem informaç,NaN,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,Universidade Federal do Pará,15000,Ministério da Educação,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,Universidade Federal do Pará,15000,Ministério da Educação,2,Cargo,ATIVO PERMANENTE,NaN,NaN,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,01/03/2013,NaN,24/07/2006,2475,16/08/2006,NaN,PORTARIA,PORTARIA,-1,True
29,310314810,ABADIO DOS REIS SILVA LEITE,***.968.426-**,000****,Sem informaç,NaN,-1.0,-1,-1.0,CD,4,CARGO DE DIRECAO - CD - IFES,13,COORDENADOR(A) GERAL,S,26413000000021,CAMPUS UBERABA-DIRECAO GERAL,26413,Instituto Federal do Triângulo Mineiro,15000,Ministério da Educação,26413000000287,COORD GERAL ADM PLANEJAMENTO-URA,26413,Instituto Federal do Triângulo Mineiro,15000,Ministério da Educação,1,Função,ATIVO PERMANENTE,NaN,NaN,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,26/12/2019,NaN,29/12/2008,107,23/01/1984,NaN,LEI,PORTARIA,-1,True
30,310314810,ABADIO DOS REIS SILVA LEITE,***.968.426-**,000****,ASSISTENTE EM ADMINISTRACAO,D,0.0,416,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,26413000000021,CAMPUS UBERABA-DIRECAO GERAL,26413,Instituto Federal do Triângulo Mineiro,15000,Ministério da Educação,26413000000287,COORD GERAL ADM PLANEJAMENTO-URA,26413,Instituto Federal do Triângulo Mineiro,15000,Ministério da Educação,2,Cargo,ATIVO PERMANENTE,NaN,NaN,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/03/2005,NaN,29/12/2008,107,23/01/1984,NaN,LEI,PORTARIA,-1,True
40,311514928,ABDALLA MAHAMAD ABDO,***.321.901-**,012****,Sem informaç,NaN,-1.0,-1,-1.0,GF,25,FUNCAO GRATIFICADA,1270,ENCARREGADO SETOR/SUREG GF-IV,S,22200027305300,SETOR DE PROGR INSTIT E SOCIAIS DE ABAST,22200,Companhia Nacional de Abastecimento,13100,MIN DO DESENV AGR E AGRIC FAMILIAR,-3,Inválido,22200,Companhia Nacional de Abastecimento,13100,MIN DO DESENV AGR E AGRIC FAMILIAR,1,Função,CELETISTA/EMPREGADO,NaN,NaN,CONSOLIDACAO DAS LEIS DO TRABALHO,40 HORAS SEMANAIS,01/06/2016,NaN,28/05/1987,NaN,NaN,NaN,CONTRATO,Inválido,-1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711974,310551429,ZULMIRA NOBREGA PIVA DE CARVALHO,***.023.724-**,013****,PROFESSOR DO MAGISTERIO SUPERIOR,7,0.0,NaN,703.0,-1,-1,Sem informação,-1,Sem informaç,NaN,26240000001036,CCTA-DEPARTAMENTO DE JORNALISMO,26240,Universidade Federal da Paraíba,15000,Ministério da Educação,26240000000633,PROGRAMA DE POS EM JORNALISMO,26240,Universidade Federal da Paraíba,15000,Ministério da Educação,2,Carg

In [17]:
## df[(df['duplicado_id']==True) & (df['Id_SERVIDOR_PORTAL']!=-11)]['DESCRICAO_CARGO'].unique()

array(['Sem informaç', 'PROFESSOR DO MAGISTERIO SUPERIOR',
       'ASSISTENTE EM ADMINISTRACAO', ...,
       'PROIND - DESENHISTA PROJETISTA', 'PROSAL - COMPRADOR',
       'RESIDENCIA MULTIPROF. SAUDE DA FAMILIA'], dtype=object)

In [18]:
df['DESCRICAO_CARGO'].unique()

array(['BIBLIOTECARIO-DOCUMENTALISTA', 'Sem informaç',
       'PROFESSOR DO MAGISTERIO SUPERIOR', ...,
       'AUXILIAR DE APOIO OPERACIONAL III', 'ANALISTA DE EQUIPAMENTOS',
       'Sigiloso'], dtype=object)

In [58]:
## df[(df['duplicado_id']==True)&(df['Id_SERVIDOR_PORTAL']!=-11)&(df['TIPO_VINCULO']=="Outros")]

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,DESCRICAO_CARGO,CLASSE_CARGO,REFERENCIA_CARGO,PADRAO_CARGO,NIVEL_CARGO,SIGLA_FUNCAO,NIVEL_FUNCAO,FUNCAO,CODIGO_ATIVIDADE,ATIVIDADE,OPCAO_PARCIAL,COD_UORG_LOTACAO,UORG_LOTACAO,COD_ORG_LOTACAO,ORG_LOTACAO,COD_ORGSUP_LOTACAO,ORGSUP_LOTACAO,COD_UORG_EXERCICIO,UORG_EXERCICIO,COD_ORG_EXERCICIO,ORG_EXERCICIO,COD_ORGSUP_EXERCICIO,ORGSUP_EXERCICIO,COD_TIPO_VINCULO,TIPO_VINCULO,SITUACAO_VINCULO,DATA_INICIO_AFASTAMENTO,DATA_TERMINO_AFASTAMENTO,REGIME_JURIDICO,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,UF_EXERCICIO,duplicado_id
50,312476777,ABDENEGO ALBINO GOMES,***.880.034-**,018****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,49200,MINISTERIO DOS TRANSPORTES,20101,Presidência da República,26418000000017,DIRETORIA DE ADMINISTRACAO/RECIFE,26418,Instituto Federal de Pernambuco,15000,Ministério da Educação,3,Outros,EXERC. 7 ART93 8112,NaN,NaN,CONSOLIDACAO DAS LEIS DO TRABALHO,40 HORAS SEMANAIS,NaN,NaN,26/07/2011,8878,05/08/2011,NaN,PARECER,LEI,-1,True
73,311048237,ABDO HALLACK,***.580.636-**,026****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,17500,MIN GESTAO E INOV EM SERV PUBLICOS,-1,Sem informação,39252000001529,SERVICO UNIDADE LOCAL JUIZ DE FORA - MG,39252,Departamento Nacional de Infraestrutura de Tra...,49200,MINISTERIO DOS TRANSPORTES,3,Outros,EXERC DESCENT CARREI,NaN,NaN,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,NaN,NaN,28/11/2012,260,22/12/2008,NaN,PORTARIA,PORTARIA,-1,True
131,310964901,ABEL CAMPOS DE FREITAS,***.315.367-**,010****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,15000,Ministério da Educação,-1,Sem informação,26256000000108,UNED NOVA IGUACU,26256,Centro Federal de Educação Tecnológica Celso S...,15000,Ministério da Educação,3,Outros,EXERC. 7 ART93 8112,NaN,NaN,CONSOLIDACAO DAS LEIS DO TRABALHO,40 HORAS SEMANAIS,NaN,NaN,11/03/2013,53,29/04/2013,NaN,PORTARIA,PORTARIA,-1,True
160,313102398,ABEL FERREIRA LOPES FILHO,***.356.823-**,034****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,33000,MINISTERIO DO TRABALHO E PREVIDENCIA,-1,Sem informação,20101000002772,DIRETORIA ACOMP JUNTO CONGRESSO NACIONAL,20101,Presidência da República,-1,Sem informação,3,Outros,CEDIDO/REQUISITADO,NaN,NaN,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,NaN,NaN,08/09/2023,00001167,03/11/2004,NaN,PORTARIA,PORTARIA,-1,True
161,310388898,ABEL FLAVIO DOS REIS,***.812.336-**,019****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,32396,Agência Nacional de Mineração,32000,Ministério de Minas e Energia,26261000000293,COORDENACAO DE EXTENSAO - CI,26261,Universidade Federal de Itajubá - MG,15000,Ministério da Educação,3,Outros,EXERC. 7 ART93 8112,NaN,NaN,CONSOLIDACAO DAS LEIS DO TRABALHO,40 HORAS SEMANAIS,NaN,NaN,13/01/2022,2408,06/12/2011,NaN,PORTARIA,PORTARIA,-1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711844,310371110,ZULEICA ALVES NONATO VINTE,***.077.576-**,001****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,17600,MINISTERIO DA FAZENDA,20101,Presidência da República,17500000002642,DIVISAO DE ADMINISTRACAO E LOGISTICA,17500,MIN GESTAO E INOV EM SERV PUBLICOS,-1,Sem informação,3,Outros,CEDIDO/REQUISITADO,NaN,NaN,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,NaN,NaN,21/06/2023,000000011,04/05/1981,NaN,LEI,EXP. MOTIVOS,-1,True
711887,313109108,ZULEIDE FERREIRA FILGUEIRAS,***.485.946-**,013****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,45205,Fundação Instituto Brasileiro de Geografia e E...,17300,MINISTERIO DO PLANEJAMENTO E ORCAMENTO,37000000000025,DPU - MINAS G

In [55]:
## df[(df['TIPO_VINCULO']=="Outros")&(df['Id_SERVIDOR_PORTAL']!=-11)]

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,DESCRICAO_CARGO,CLASSE_CARGO,REFERENCIA_CARGO,PADRAO_CARGO,NIVEL_CARGO,SIGLA_FUNCAO,NIVEL_FUNCAO,FUNCAO,CODIGO_ATIVIDADE,ATIVIDADE,OPCAO_PARCIAL,COD_UORG_LOTACAO,UORG_LOTACAO,COD_ORG_LOTACAO,ORG_LOTACAO,COD_ORGSUP_LOTACAO,ORGSUP_LOTACAO,COD_UORG_EXERCICIO,UORG_EXERCICIO,COD_ORG_EXERCICIO,ORG_EXERCICIO,COD_ORGSUP_EXERCICIO,ORGSUP_EXERCICIO,COD_TIPO_VINCULO,TIPO_VINCULO,SITUACAO_VINCULO,DATA_INICIO_AFASTAMENTO,DATA_TERMINO_AFASTAMENTO,REGIME_JURIDICO,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,UF_EXERCICIO,duplicado_id
9,313302667,AARON MATHEUS MOURA,***.470.823-**,033****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,17600,MINISTERIO DA FAZENDA,20101,Presidência da República,17600000002916,PROC ESTADUAL DA FAZ NAC DO CEARA,17600,MINISTERIO DA FAZENDA,20101,Presidência da República,3,Outros,ESTAGIARIO SIGEPE,NaN,NaN,ESTAGIO,20 HORAS SEMANAIS,NaN,NaN,18/12/2019,NaN,NaN,NaN,PORTARIA,Inválido,-1,False
50,312476777,ABDENEGO ALBINO GOMES,***.880.034-**,018****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,49200,MINISTERIO DOS TRANSPORTES,20101,Presidência da República,26418000000017,DIRETORIA DE ADMINISTRACAO/RECIFE,26418,Instituto Federal de Pernambuco,15000,Ministério da Educação,3,Outros,EXERC. 7 ART93 8112,NaN,NaN,CONSOLIDACAO DAS LEIS DO TRABALHO,40 HORAS SEMANAIS,NaN,NaN,26/07/2011,8878,05/08/2011,NaN,PARECER,LEI,-1,True
73,311048237,ABDO HALLACK,***.580.636-**,026****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,17500,MIN GESTAO E INOV EM SERV PUBLICOS,-1,Sem informação,39252000001529,SERVICO UNIDADE LOCAL JUIZ DE FORA - MG,39252,Departamento Nacional de Infraestrutura de Tra...,49200,MINISTERIO DOS TRANSPORTES,3,Outros,EXERC DESCENT CARREI,NaN,NaN,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,NaN,NaN,28/11/2012,260,22/12/2008,NaN,PORTARIA,PORTARIA,-1,True
76,310822592,ABDON BANDEIRA ANDRE FILHO,***.884.724-**,033****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,26240,Universidade Federal da Paraíba,15000,Ministério da Educação,26240000000137,CENTRO DE TECNOLOGIA,26240,Universidade Federal da Paraíba,15000,Ministério da Educação,3,Outros,ESTAGIARIO SIGEPE,NaN,NaN,ESTAGIO,20 HORAS SEMANAIS,NaN,NaN,14/02/2023,NaN,NaN,NaN,NORMA COMPLEMENTAR,Inválido,-1,False
119,311300769,ABEL APARECIDO DA SILVA,***.733.898-**,010****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,99011,Empresa Brasileira de Infraestrutura Aeroportu...,49000,Ministério da Infraestrutura,40116000000898,SETOR - PROC SEC FEDERAL EM JUNDIAI,40116,ADVOCACIA-GERAL DA UNIAO,20101,Presidência da República,3,Outros,REQ.DE OUTROS ORGAOS,NaN,NaN,CONSOLIDACAO DAS LEIS DO TRABALHO,40 HORAS SEMANAIS,NaN,NaN,01/01/2023,SN,01/12/2000,NaN,DECRETO,CONTRATO,-1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711905,312700621,ZULEIDE RAMOS DE OLIVEIRA,***.156.223-**,001****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,17600,MINISTERIO DA FAZENDA,20101,Presidência da República,17500000002747,"COORD DE GEST PUBL, FREQ E ASSENT FUNC",17500,MIN GESTAO E INOV EM SERV PUBLICOS,-1,Sem informação,3,Outros,CEDIDO/REQUISITADO,NaN,NaN,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,NaN,NaN,21/06/2023,000000422,05/07/1985,NaN,LEI,PORTARIA,-1,True
711942,313225197,ZULMA ARAUJO COURY,***.167.441-**,016****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,13000,"Ministério da Agricultura, Pecuária e Abasteci...",-1,Sem informação,17600000003112,COORDENACAO DE GESTAO DE PESSOAS,17600,MINISTERIO DA FAZENDA,20101,Presidênci

In [40]:
df['TIPO_VINCULO'].value_counts()

Cargo     608371
Função     97120
Outros     43105
Name: TIPO_VINCULO, dtype: int64

In [19]:
np.set_options(threshold=748596)
print(df['DESCRICAO_CARGO'].unique())

##CONTRA REGRA, Sem informaç, Inválido

['BIBLIOTECARIO-DOCUMENTALISTA' 'Sem informaç'
 'PROFESSOR DO MAGISTERIO SUPERIOR' 'FISIOTERAPEUTA - 30H'
 'PROFESSOR ENS BASICO TECN TECNOLOGICO' 'AGENTE DE COMBATE AS ENDEMIAS'
 'AGENTE ADMINISTRATIVO' 'QUIMICO' 'Inválido'
 'AGENTE METROV - CONTROLE DE ESTACAO' 'MEDICO PROGRAMA MAIS MEDICO'
 'ASSISTENTE EM ADMINISTRACAO' 'AUXILIAR EM ENFERMAGEM'
 'AUXILIAR OPERACIONAL SERV DIVERSOS - NA' 'ASSISTENTE ADMINISTRATIVO'
 'TECNICO EM ENFERMAGEM' 'AGENTE DE PORTARIA' 'TECNICO DE ENFERMAGEM'
 'AUXILIAR DE ENFERMAGEM' 'TECNICO EM ENFERMAGEM - 36H'
 'ARTIFICE DE MECANICA' 'GUARDA DE ENDEMIAS' 'MEDICO - 24H'
 'AGENTE DE VIGILANCIA' 'MEDICO RESIDENTE' 'CONTRATADO LEI 8745-93 - NI'
 'ATO - ASSISTENTE DE OPERACOES' 'TEC EM ASSUNTOS EDUCACIONAIS'
 'OPERADOR DE MAQ DE VIA TERRAPLANAGEM' 'TGE-TECNICO DE GESTAO'
 'DATILOGRAFO' 'ADMINISTRADOR' 'ANALISTA TRIBUTARIO REC FEDERAL BRASIL'
 'CONTADOR' 'PINTOR-AREA' 'TECNICO DE LABORATORIO AREA'
 'TECNICO EM ELETROMECANICA' 'AUDITOR'
 'ESPECIALISTA EM INFRA-E

In [33]:
pd.options.display.max_columns = 100

In [44]:
base

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,DESCRICAO_CARGO,CLASSE_CARGO,REFERENCIA_CARGO,PADRAO_CARGO,NIVEL_CARGO,SIGLA_FUNCAO,NIVEL_FUNCAO,FUNCAO,CODIGO_ATIVIDADE,ATIVIDADE,OPCAO_PARCIAL,COD_UORG_LOTACAO,UORG_LOTACAO,COD_ORG_LOTACAO,ORG_LOTACAO,COD_ORGSUP_LOTACAO,ORGSUP_LOTACAO,COD_UORG_EXERCICIO,UORG_EXERCICIO,COD_ORG_EXERCICIO,ORG_EXERCICIO,COD_ORGSUP_EXERCICIO,ORGSUP_EXERCICIO,COD_TIPO_VINCULO,TIPO_VINCULO,SITUACAO_VINCULO,DATA_INICIO_AFASTAMENTO,DATA_TERMINO_AFASTAMENTO,REGIME_JURIDICO,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,UF_EXERCICIO
0,310606473,AARAO CARLOS LUZ MACAMBIRA,***.017.623-**,016****,BIBLIOTECARIO-DOCUMENTALISTA,E,0.0,410,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,26405000000040,DIRETORIA GERAL/CAMPUS SOBRAL,26405,Instituto Federal do Ceará,15000,Ministério da Educação,26405000000717,DIRETORIA DE ENSINO-SOB,26405,Instituto Federal do Ceará,15000,Ministério da Educação,2,Cargo,ATIVO PERMANENTE,NaN,NaN,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,11/02/2009,NaN,29/12/2008,699,11/02/2009,NaN,LEI,PORTARIA,-1
1,313006968,AARAO FERREIRA LIMA NETO,***.116.132-**,014****,Sem informaç,NaN,-1.0,-1,-1.0,FG,1,FUNCAO GRATIFICADA - IFES,99,DIRETOR(A),NaN,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,Universidade Federal do Pará,15000,Ministério da Educação,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,Universidade Federal do Pará,15000,Ministério da Educação,1,Função,ATIVO PERMANENTE,NaN,NaN,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,16/03/2020,NaN,24/07/2006,2475,16/08/2006,NaN,PORTARIA,PORTARIA,-1
2,313006968,AARAO FERREIRA LIMA NETO,***.116.132-**,014****,PROFESSOR DO MAGISTERIO SUPERIOR,7,0.0,NaN,702.0,-1,-1,Sem informação,-1,Sem informaç,NaN,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,Universidade Federal do Pará,15000,Ministério da Educação,26239000001515,NUCLEO DE DESENV AMAZONICO EM ENGENHARIA,26239,Universidade Federal do Pará,15000,Ministério da Educação,2,Cargo,ATIVO PERMANENTE,NaN,NaN,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,01/03/2013,NaN,24/07/2006,2475,16/08/2006,NaN,PORTARIA,PORTARIA,-1
3,311039523,AARAO MEIR SERRUYA,***.693.832-**,033****,FISIOTERAPEUTA - 30H,S,0.0,NaN,201.0,-1,-1,Sem informação,-1,Sem informaç,NaN,26443035000000,COMPLEXO HOSPITALAR DA UFPA,26443,Empresa Brasileira de Serviços Hospitalares,15000,Ministério da Educação,-3,Inválido,26443,Empresa Brasileira de Serviços Hospitalares,15000,Ministério da Educação,2,Cargo,CELETISTA/EMPREGADO,NaN,NaN,CONSOLIDACAO DAS LEIS DO TRABALHO,30 HORAS SEMANAIS,07/12/2022,NaN,07/12/2022,NaN,NaN,NaN,CONTRATO,Inválido,-1
4,310150191,AARAO PEREIRA DE ARAUJO JUNIOR,***.031.184-**,002****,PROFESSOR ENS BASICO TECN TECNOLOGICO,D,0.0,NaN,501.0,-1,-1,Sem informação,-1,Sem informaç,NaN,26417000000013,DIR. DESENVOLVIMENTO ENSINO-JP,26417,Instituto Federal da Paraíba,15000,Ministério da Educação,26417000000240,"UNID. ACAD. I DES, ESTR.M.AMB-JP",26417,Instituto Federal da Paraíba,15000,Ministério da Educação,2,Cargo,ATIVO PERMANENTE,NaN,NaN,REGIME JURIDICO UNICO,DEDICACAO EXCLUSIVA,01/03/2013,NaN,29/12/2008,000000124,03/05/1993,NaN,LEI,PORTARIA,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748591,-11,Sigiloso,NaN,001****,Sigiloso,NaN,NaN,NaN,NaN,-11,-11,Sigiloso,-11,Sigiloso,N,-11,Sigiloso,30802,Departamento de Polícia Rodoviária Federal,20000,Ministério da Justiça e Segurança Pública,-11,Sigiloso,30802,Departamento de Polícia Rodoviária Federal,20000,Ministério da Justiça e Segurança Pública,2,Cargo,Sigiloso,NaN,NaN,Sem informaç,Sem informaç,NaN,NaN,NaN,NaN,NaN,NaN,Sem informaç,Sem informaç,-1
748592,-11,Sigiloso,NaN,001****,Sigiloso,NaN,NaN,NaN,NaN,-11,-11,Sigiloso,-11,Sigiloso,N,-11,Sigil

In [100]:
colunas = df.columns

In [ ]:
(df['DESCRICAO_CARGO']!=[ 'Sem informaç', 'Inválido']) # mudar pra codigo do cargo


In [86]:
servidor = df['Id_SERVIDOR_PORTAL'].unique()
servidor

array([313006968, 310314810, 311514928, 312476777, 313140258, 312478428,
       310374200, 312530598, 311048237, 313315236, 310499742, 312714336,
       312785302, 313052969, 310004464, 312728147, 312562351, 313082961,
       313083005, 310964901, 310000148, 312560571, 312645075, 313102398,
       310388898, 312496882, 311532486, 312668600, 312496663, 312624931,
       310625091, 311014916, 311091532, 312542664, 313209795, 310026983,
       309908372, 310463961, 309975371, 312984207, 311582005, 310557425,
       309886480, 309948975, 310512888, 313151884, 312532744, 311121536,
       310880849, 312570371, 310923714, 312571858, 311204405, 309920107,
       310092570, 310147236, 311403308, 312712901, 310807484, 310125391,
       313088151, 312679220, 313051748, 311943713, 312580267, 310544542,
       310999115, 309916324, 310313454, 313199533, 313433699, 310354490,
       310330471, 309901908, 312878104, 310297955, 310777883, 310404073,
       310097017, 310734415, 313110803, 311185055, 

In [12]:
df1 = pd.pivot_table(df, values='NOME', index = 'Id_SERVIDOR_PORTAL', columns='TIPO_VINCULO', aggfunc='count')
y = pd.DataFrame(df1)
df1 = y.reset_index()
df1

TIPO_VINCULO,Id_SERVIDOR_PORTAL,Cargo,Função,Outros
0,309811901,1.0,1.0,NaN
1,309813757,1.0,1.0,NaN
2,309813821,1.0,1.0,NaN
3,309816675,2.0,NaN,NaN
4,309816765,1.0,1.0,NaN
...,...,...,...,...
98395,313636920,1.0,1.0,NaN
98396,313639032,1.0,1.0,NaN
98397,313641144,1.0,1.0,NaN
98398,313666766,2.0,NaN,NaN


In [23]:
df.columns

Index(['Id_SERVIDOR_PORTAL', 'NOME', 'CPF', 'MATRICULA', 'DESCRICAO_CARGO',
       'CLASSE_CARGO', 'REFERENCIA_CARGO', 'PADRAO_CARGO', 'NIVEL_CARGO',
       'SIGLA_FUNCAO', 'NIVEL_FUNCAO', 'FUNCAO', 'CODIGO_ATIVIDADE',
       'ATIVIDADE', 'OPCAO_PARCIAL', 'COD_UORG_LOTACAO', 'UORG_LOTACAO',
       'COD_ORG_LOTACAO', 'ORG_LOTACAO', 'COD_ORGSUP_LOTACAO',
       'ORGSUP_LOTACAO', 'COD_UORG_EXERCICIO', 'UORG_EXERCICIO',
       'COD_ORG_EXERCICIO', 'ORG_EXERCICIO', 'COD_ORGSUP_EXERCICIO',
       'ORGSUP_EXERCICIO', 'COD_TIPO_VINCULO', 'TIPO_VINCULO',
       'SITUACAO_VINCULO', 'DATA_INICIO_AFASTAMENTO',
       'DATA_TERMINO_AFASTAMENTO', 'REGIME_JURIDICO', 'JORNADA_DE_TRABALHO',
       'DATA_INGRESSO_CARGOFUNCAO', 'DATA_NOMEACAO_CARGOFUNCAO',
       'DATA_INGRESSO_ORGAO', 'DOCUMENTO_INGRESSO_SERVICOPUBLICO',
       'DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO', 'DIPLOMA_INGRESSO_CARGOFUNCAO',
       'DIPLOMA_INGRESSO_ORGAO', 'DIPLOMA_INGRESSO_SERVICOPUBLICO',
       'UF_EXERCICIO', 'duplicado_id']

In [26]:
pd.set_option('display.max_columns', None)

In [38]:
remunera[remunera['Id_SERVIDOR_PORTAL']==310398462]

,ANO,MES,Id_SERVIDOR_PORTAL,CPF,NOME,REMUNERAÇÃO BÁSICA BRUTA (R$),REMUNERAÇÃO BÁSICA BRUTA (U$),ABATE-TETO (R$),ABATE-TETO (U$),GRATIFICAÇÃO NATALINA (R$),GRATIFICAÇÃO NATALINA (U$),ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$),ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$),FÉRIAS (R$),FÉRIAS (U$),OUTRAS REMUNERAÇÕES EVENTUAIS (R$),OUTRAS REMUNERAÇÕES EVENTUAIS (U$),IRRF (R$),IRRF (U$),PSS/RPGS (R$),PSS/RPGS (U$),DEMAIS DEDUÇÕES (R$),DEMAIS DEDUÇÕES (U$),PENSÃO MILITAR (R$),PENSÃO MILITAR (U$),FUNDO DE SAÚDE (R$),FUNDO DE SAÚDE (U$),TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$),TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$),REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$),REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*),VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$),VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$),TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*),TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)
123665,2023,11.0,310398462.0,***.796.001-**,DJACI VIEIRA DE SOUSA,"13946,90","0,00","0,00","0,00","15413,90","0,00","0,00","0,00","0,00","0,00","0,00","0,00","-5567,98","0,00","-1267,60","0,00","-6973,45","0,00","0,00","0,00","0,00","0,00","0,00","0,00","15551,77","0,00","1355,73","0,00","0,00","0,00","0,00","0,00","1355,73","0,00"


In [37]:
df[df['Id_SERVIDOR_PORTAL']==310398462]

,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,DESCRICAO_CARGO,CLASSE_CARGO,REFERENCIA_CARGO,PADRAO_CARGO,NIVEL_CARGO,SIGLA_FUNCAO,NIVEL_FUNCAO,FUNCAO,CODIGO_ATIVIDADE,ATIVIDADE,OPCAO_PARCIAL,COD_UORG_LOTACAO,UORG_LOTACAO,COD_ORG_LOTACAO,ORG_LOTACAO,COD_ORGSUP_LOTACAO,ORGSUP_LOTACAO,COD_UORG_EXERCICIO,UORG_EXERCICIO,COD_ORG_EXERCICIO,ORG_EXERCICIO,COD_ORGSUP_EXERCICIO,ORGSUP_EXERCICIO,COD_TIPO_VINCULO,TIPO_VINCULO,SITUACAO_VINCULO,DATA_INICIO_AFASTAMENTO,DATA_TERMINO_AFASTAMENTO,REGIME_JURIDICO,JORNADA_DE_TRABALHO,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,UF_EXERCICIO,duplicado_id
172748,310398462,DJACI VIEIRA DE SOUSA,***.796.001-**,011****,Sem informaç,NaN,-1.0,-1,-1.0,FEX,215,FUNCÃO COMISSIONADA EXECUTIVA,50,ASSESSOR(A) ESPECIAL,NaN,17500000003375,UORG DE CONTROLE DE SERVID ADM DO MGI,17500,MIN GESTAO E INOV EM SERV PUBLICOS,-1,Sem informação,17400000000001,MIN DESENVOLV IND COMERCIO E SERVICOS,17400,MIN DESENVOLV IND COMERCIO E SERVICOS,-1,Sem informação,1,Função,CEDIDO/REQUISITADO,NaN,NaN,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/07/2023,NaN,19/06/2023,000000002,12/01/1995,NaN,LEI,PORTARIA,-1,True
172749,310398462,DJACI VIEIRA DE SOUSA,***.796.001-**,021****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,99013,Hospital de Clínicas de Porto Alegre,15000,Ministério da Educação,99013150000000,COORDENADORIA DE GESTAO DE PESSOAS,99013,Hospital de Clínicas de Porto Alegre,15000,Ministério da Educação,3,Outros,SEM VINCULO,NaN,NaN,CONSOLIDACAO DAS LEIS DO TRABALHO,24 HORAS SEMANAIS,NaN,NaN,26/04/2021,NaN,NaN,NaN,CONTRATO,Inválido,-1,True
172750,310398462,DJACI VIEIRA DE SOUSA,***.796.001-**,011****,AGENTE ADMINISTRATIVO,S,0.0,III,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,17500000003375,UORG DE CONTROLE DE SERVID ADM DO MGI,17500,MIN GESTAO E INOV EM SERV PUBLICOS,-1,Sem informação,17400000000001,MIN DESENVOLV IND COMERCIO E SERVICOS,17400,MIN DESENVOLV IND COMERCIO E SERVICOS,-1,Sem informação,2,Cargo,ATIVO EM OUTRO ORGAO,NaN,NaN,REGIME JURIDICO UNICO,40 HORAS SEMANAIS,01/07/2006,NaN,01/01/2023,000000002,12/01/1995,NaN,OFICIO,PORTARIA,-1,True
172751,310398462,DJACI VIEIRA DE SOUSA,***.796.001-**,011****,Inválido,NaN,0.0,NaN,0.0,-1,-1,Sem informação,-1,Sem informaç,NaN,-3,Inválido,26443,Empresa Brasileira de Serviços Hospitalares,15000,Ministério da Educação,26443001000000,EMPRESA BRAS. DE SERVICOS HOSPITALARES,26443,Empresa Brasileira de Serviços Hospitalares,15000,Ministério da Educação,3,Outros,SEM VINCULO,NaN,NaN,CONSOLIDACAO DAS LEIS DO TRABALHO,Inválido,NaN,NaN,28/05/2020,NaN,NaN,NaN,ATA,Inválido,-1,True


In [28]:
remuneracao = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados novos\\CGU\\SIAPE\\SERVIDORES_ATIVOS_REMUNERACAO'

In [29]:
os.chdir(remuneracao)

In [31]:
remunera = pd.read_csv('2023_nov_siape_servidores_remuneracao.csv', encoding='latin-1', sep=';')

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_8316\1081935413.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  remunera = pd.read_csv('2023_nov_siape_servidores_remuneracao.csv', encoding='latin-1', sep=';')


In [34]:
df1[(df1['Outros']==2)]

TIPO_VINCULO,Id_SERVIDOR_PORTAL,Cargo,Função,Outros
4363,309954939,2.0,NaN,2.0
22604,310398462,1.0,1.0,2.0
41920,310927220,NaN,NaN,2.0
46081,311042510,1.0,1.0,2.0
48091,311102910,2.0,NaN,2.0
50099,311165107,1.0,NaN,2.0
51892,311226811,2.0,NaN,2.0
56925,311432323,1.0,NaN,2.0
58096,311484078,NaN,NaN,2.0
59647,311572782,NaN,NaN,2.0


In [33]:
df1['Outros'].value_counts() ## Se atentar para quem tem 2

1.0    10830
2.0       11
Name: Outros, dtype: int64

In [20]:
df1['Cargo'].value_counts() ## Se atentar para quem tem 3 e 2. 

1.0    93292
2.0     5071
3.0        3
Name: Cargo, dtype: int64

In [189]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98400 entries, 0 to 98399
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id_SERVIDOR_PORTAL  98400 non-null  int64  
 1   Cargo               98366 non-null  float64
 2   Função              84053 non-null  float64
 3   Outros              10841 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 3.0 MB


In [19]:
pd.pivot_table(df1, values='Id_SERVIDOR_PORTAL', index = 'Cargo', columns='Cargo',aggfunc='count',dropna=False,observed=False)

Cargo,1.0,2.0,3.0
Função,,,
1.0,82627.0,1392.0,NaN
2.0,NaN,3.0,NaN


In [146]:
df1[(df1['Cargo']==2.0) & (df1['Função']==2.0)]

TIPO_VINCULO,Cargo,Função,Outros
Id_SERVIDOR_PORTAL,,,
310279423,2.0,2.0,NaN
313218196,2.0,2.0,NaN
313392336,2.0,2.0,NaN


In [153]:
df1[(df1['Cargo']==3.0)]

TIPO_VINCULO,Cargo,Função,Outros
Id_SERVIDOR_PORTAL,,,
312807430,3.0,NaN,NaN
312813403,3.0,NaN,NaN
312901010,3.0,NaN,NaN


In [165]:
df1[(df1['Cargo']==2.0)]

TIPO_VINCULO,Cargo,Função,Outros
Id_SERVIDOR_PORTAL,,,
309816675,2.0,NaN,NaN
309817379,2.0,1.0,NaN
309824893,2.0,1.0,NaN
309832785,2.0,NaN,NaN
309833149,2.0,NaN,NaN
...,...,...,...
313545772,2.0,NaN,NaN
313545949,2.0,NaN,NaN
313551857,2.0,NaN,NaN


In [164]:
df1[(df1['Função']==2.0)]

TIPO_VINCULO,Cargo,Função,Outros
Id_SERVIDOR_PORTAL,,,
310279423,2.0,2.0,NaN
313218196,2.0,2.0,NaN
313392336,2.0,2.0,NaN


In [ ]:
df1[(df1['Função']==2.0)]

In [161]:
df1['Outros'].max()

2.0

In [162]:
df1['Função'].max()

2.0

In [163]:
df1['Cargo'].max()

3.0